In [1]:
import pandas as pd
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams
import time

# Get 2023-24 season
print("Fetching 2023-24 season...")
games_finder_24 = leaguegamefinder.LeagueGameFinder(
    season_nullable='2023-24',
    timeout=120
)
games_24 = games_finder_24.get_data_frames()[0]
print(f"2023-24: {len(games_24)} records")
time.sleep(2)

# Get 2022-23 season
print("Fetching 2022-23 season...")
games_finder_23 = leaguegamefinder.LeagueGameFinder(
    season_nullable='2022-23',
    timeout=120
)
games_23 = games_finder_23.get_data_frames()[0]
print(f"2022-23: {len(games_23)} records")
time.sleep(2)

# Get 2021-22 season
print("Fetching 2021-22 season...")
games_finder_22 = leaguegamefinder.LeagueGameFinder(
    season_nullable='2021-22',
    timeout=120
)
games_22 = games_finder_22.get_data_frames()[0]
print(f"2021-22: {len(games_22)} records")
time.sleep(2)

# Get 2020-21 season
print("Fetching 2020-21 season...")
games_finder_21 = leaguegamefinder.LeagueGameFinder(
    season_nullable='2020-21',
    timeout=120
)
games_21 = games_finder_21.get_data_frames()[0]
print(f"2020-21: {len(games_21)} records")

# Concatenate all seasons
all_games = pd.concat([games_21, games_22, games_23, games_24], ignore_index=True)
print(f"\nTotal combined records: {len(all_games)}")

# Get unique game IDs sorted
unique_game_ids = sorted(all_games['GAME_ID'].dropna().unique())  # There are NAs to drop
print(f"Unique games: {len(unique_game_ids)}")

# Write to CSV
all_games.to_csv('nba_games_2021_to_2024.csv', index=False)
print("Written to 'nba_games_2021_to_2024.csv'")

# Write unique game IDs to separate file
game_ids_df = pd.DataFrame({'GAME_ID': unique_game_ids})
game_ids_df['GAME_ID'] = game_ids_df['GAME_ID'].astype(str)  # ensure string since there are zeros at the front
game_ids_df.to_csv('unique_game_ids.csv', index=False)
print("Written unique game IDs to 'unique_game_ids.csv'")

# Print Statements
print(f"Total records: {len(all_games)}")
print(f"Unique games: {len(unique_game_ids)}")
print(f"First game ID: {unique_game_ids[0]}")
print(f"Last game ID: {unique_game_ids[-1]}")

Fetching 2023-24 season...
2023-24: 4385 records
Fetching 2022-23 season...
2022-23: 4322 records
Fetching 2021-22 season...
2021-22: 4182 records
Fetching 2020-21 season...
2020-21: 2726 records

Total combined records: 15615
Unique games: 7807
Written to 'nba_games_2021_to_2024.csv'
Written unique game IDs to 'unique_game_ids.csv'
Total records: 15615
Unique games: 7807
First game ID: 0012000001
Last game ID: 2072100016


With the NBA API it was a trial and error process to pull data. It did not seem like there was a definitive reason why data was not pulling. So we built a script that would simply run multiple passes over a unique games list and try to get a complete set of data. This proved to be successful. Even testing different sleep times proved to be unsuccessful.

Basically we were collecting two sets of data one was from "boxscoretraditionalv2" which gave team and player stats. We needed to collect a set for the first half and the complete game this would give us a complete picture of the data so we could predict a second half point total. We also used "boxscoresummaryv2" to collect game times, referees, injuries, points by qtr. It would basically pull a list of 7 tables, the multiple pass code was neccesarily for this as it seemed to fail to pull more often. In addition we had to patch this API pull to code in 'GAME_ID' for a couple of the tables that did not have it present.

In [ ]:
# Getting First Half Box Score Data - Multi-Pass Version
from nba_api.stats.endpoints import boxscoretraditionalv2
import pandas as pd
import time
import os

# Configuration - Change these for each pass
PASS_NUMBER = 3
INPUT_FILE = 'unique_game_ids.csv' if PASS_NUMBER == 1 else f'failed_game_ids_fh_pass{PASS_NUMBER-1}.csv'
OUTPUT_SUFFIX = '' if PASS_NUMBER == 1 else f'_pt{PASS_NUMBER}'

# Loading Game IDs
game_ids_df = pd.read_csv(INPUT_FILE, dtype={'GAME_ID': str})
game_ids_list = game_ids_df['GAME_ID'].tolist()

# Initial Print Statements
print(f"=== PASS {PASS_NUMBER} ===")
print(f"Total games to process: {len(game_ids_list)}")
print(f"First game ID: {game_ids_list[0]}")
print(f"Last game ID: {game_ids_list[-1]}")

# Lists to collect dataframes
all_fh_player_stats = []
all_fh_team_stats = []
all_fh_starter_bench = []

# Variables to track progress
games_processed = 0
games_failed = 0
start_time = time.time()
failed_game_ids = []

# First Half Loop
for idx, game_id in enumerate(game_ids_list):
    try:
        # API function
        fh = boxscoretraditionalv2.BoxScoreTraditionalV2(
            game_id=game_id,
            range_type=1,
            start_period=1,
            end_period=2
        )
        
        # Frames to DF
        dfs = fh.get_data_frames()
        
        # Append
        all_fh_player_stats.append(dfs[0])
        all_fh_team_stats.append(dfs[1])
        all_fh_starter_bench.append(dfs[2])
        
        # Track process
        games_processed += 1
        
        # Progress Update every 1000 games
        if (idx + 1) % 1000 == 0:
            elapsed = time.time() - start_time
            print(f"Progress: {idx + 1}/{len(game_ids_list)} games ({games_processed} success, {games_failed} failed) - {elapsed/60:.1f} min elapsed")
        
        time.sleep(0.5)
        
    except Exception as e:
        games_failed += 1
        # Track failed game ID
        failed_game_ids.append(game_id)
        # Print at 1st failure and every 50
        if games_failed == 1 or games_failed % 50 == 0:
            print(f"Failures: {games_failed}")
        continue

# Combine into DFs
if all_fh_player_stats:
    fh_players_combined = pd.concat(all_fh_player_stats, ignore_index=True)
    fh_teams_combined = pd.concat(all_fh_team_stats, ignore_index=True)
    fh_starters_bench_combined = pd.concat(all_fh_starter_bench, ignore_index=True)
    
    # Save combined dataframes with suffix
    fh_players_combined.to_csv(f'first_half_players{OUTPUT_SUFFIX}.csv', index=False)
    fh_teams_combined.to_csv(f'first_half_teams{OUTPUT_SUFFIX}.csv', index=False)
    fh_starters_bench_combined.to_csv(f'first_half_starters_bench{OUTPUT_SUFFIX}.csv', index=False)
    
    print(f"Successfully processed: {games_processed}")
    print(f"Player records: {len(fh_players_combined)}")
    print(f"Team records: {len(fh_teams_combined)}")
else:
    print("\nNo data collected - all games failed")

# Save failed game IDs
if failed_game_ids:
    failed_df = pd.DataFrame({'GAME_ID': failed_game_ids})
    failed_df.to_csv(f'failed_game_ids_fh_pass{PASS_NUMBER}.csv', index=False)
    print(f"\nFailed game IDs saved to 'failed_game_ids_fh_pass{PASS_NUMBER}.csv'")
    print(f"Failed games: {len(failed_game_ids)}")
    print(f"\nTo run pass {PASS_NUMBER + 1}:")
else:
    print(f"\n All games successful - no need for pass {PASS_NUMBER + 1}!")

# Final Summary
elapsed = time.time() - start_time
print(f"PASS {PASS_NUMBER} COMPLETE")
print(f"Successfully processed: {games_processed}/{len(game_ids_list)}")
print(f"Failed: {games_failed}/{len(game_ids_list)}")
print(f"Total time: {elapsed/60:.1f} minutes")

=== PASS 3 ===
Total games to process: 301
First game ID: 0022100426
Last game ID: 2052100091

✅ Data saved!
Successfully processed: 301
Player records: 5716
Team records: 600

🎉 All games successful - no need for pass 4!

PASS 3 COMPLETE
Successfully processed: 301/301
Failed: 0/301
Total time: 20.0 minutes


/var/folders/0b/xk0_fhkn7l3czh5hp6y8gv040000gn/T/ipykernel_26975/222045344.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fh_players_combined = pd.concat(all_fh_player_stats, ignore_index=True)
/var/folders/0b/xk0_fhkn7l3czh5hp6y8gv040000gn/T/ipykernel_26975/222045344.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fh_teams_combined = pd.concat(all_fh_team_stats, ignore_index=True)
/var/folders/0b/xk0_fhkn7l3czh5hp6y8gv040000gn/T/ipykernel_26975/222045344.py:79: FutureWarning: The beh

In [ ]:
# Getting Box Score Summary Data - Multi-Pass with Auto-Loop Version (with GAME_ID fix)
from nba_api.stats.endpoints import boxscoresummaryv2
import pandas as pd
import time
import os

# Configuration
MAX_PASSES = 15
INITIAL_INPUT_FILE = 'unique_game_ids.csv'

# Start with pass 1
for pass_num in range(1, MAX_PASSES + 1):
    print(f"\n{'='*60}")
    print(f"=== STARTING PASS {pass_num} ===")
    print(f"{'='*60}\n")
    
    # Determine input file
    if pass_num == 1:
        input_file = INITIAL_INPUT_FILE
    else:
        input_file = f'failed_game_ids_bss_pass{pass_num-1}.csv'
        
        # Check if there are failures to process
        if not os.path.exists(input_file):
            print(f"✅ No failures from pass {pass_num-1} - All done!")
            break
    
    # Loading Game IDs
    game_ids_df = pd.read_csv(input_file, dtype={'GAME_ID': str})
    game_ids_list = game_ids_df['GAME_ID'].tolist()
    
    # Output suffix
    output_suffix = '' if pass_num == 1 else f'_pt{pass_num}'
    
    # Initial Print Statements
    print(f"Total games to process: {len(game_ids_list)}")
    print(f"First game ID: {game_ids_list[0]}")
    print(f"Last game ID: {game_ids_list[-1]}")
    
    # Lists to collect dataframes
    all_game_summary = []
    all_team_stats = []
    all_refs = []
    all_inactive = []
    all_game_info = []
    all_points_by_qtr = []
    all_last_meeting = []
    
    # Variables to track progress
    games_processed = 0
    games_failed = 0
    start_time = time.time()
    failed_game_ids = []
    
    # Loop through games
    for idx, game_id in enumerate(game_ids_list):
        try:
            # API function
            summary = boxscoresummaryv2.BoxScoreSummaryV2(game_id=game_id)
            
            # Get all dataframes
            dfs = summary.get_data_frames()
            
            # Add GAME_ID to each dataframe if it doesn't exist
            for i in range(7):  # Only process 0-6, skip 7-8
                if 'GAME_ID' not in dfs[i].columns:
                    dfs[i]['GAME_ID'] = game_id
            
            # Append only the ones we want (0-6, drop 7-8)
            all_game_summary.append(dfs[0])
            all_team_stats.append(dfs[1])
            all_refs.append(dfs[2])
            all_inactive.append(dfs[3])
            all_game_info.append(dfs[4])
            all_points_by_qtr.append(dfs[5])
            all_last_meeting.append(dfs[6])
            
            games_processed += 1
            
            # Progress Update every 1000 games
            if (idx + 1) % 1000 == 0:
                elapsed = time.time() - start_time
                print(f"Progress: {idx + 1}/{len(game_ids_list)} games ({games_processed} success, {games_failed} failed) - {elapsed/60:.1f} min elapsed")
            
            time.sleep(0.5)
            
        except Exception as e:
            games_failed += 1
            failed_game_ids.append(game_id)
            if games_failed == 1 or games_failed % 50 == 0:
                print(f"Failures: {games_failed}")
            continue
    
    # Combine into DFs
    if all_game_summary:
        game_summary_combined = pd.concat(all_game_summary, ignore_index=True)
        team_stats_combined = pd.concat(all_team_stats, ignore_index=True)
        refs_combined = pd.concat(all_refs, ignore_index=True)
        inactive_combined = pd.concat(all_inactive, ignore_index=True)
        game_info_combined = pd.concat(all_game_info, ignore_index=True)
        points_qtr_combined = pd.concat(all_points_by_qtr, ignore_index=True)
        last_meeting_combined = pd.concat(all_last_meeting, ignore_index=True)
        
        # Save combined dataframes
        game_summary_combined.to_csv(f'game_summary{output_suffix}.csv', index=False)
        team_stats_combined.to_csv(f'team_stats{output_suffix}.csv', index=False)
        refs_combined.to_csv(f'refs{output_suffix}.csv', index=False)
        inactive_combined.to_csv(f'inactive_players{output_suffix}.csv', index=False)
        game_info_combined.to_csv(f'game_info{output_suffix}.csv', index=False)
        points_qtr_combined.to_csv(f'points_by_quarter{output_suffix}.csv', index=False)
        last_meeting_combined.to_csv(f'last_meeting{output_suffix}.csv', index=False)
        
        print(f"\nPass {pass_num} data saved!")
        print(f"Successfully processed: {games_processed}")
    else:
        print(f"\nPass {pass_num}: No data collected - all games failed")
    
    # Save failed game IDs
    if failed_game_ids:
        failed_df = pd.DataFrame({'GAME_ID': failed_game_ids})
        failed_df.to_csv(f'failed_game_ids_bss_pass{pass_num}.csv', index=False)
        print(f"Failed game IDs saved to 'failed_game_ids_bss_pass{pass_num}.csv'")
        print(f"Failed games: {len(failed_game_ids)}")
    else:
        print(f"Pass {pass_num}: All games successful!")
    
    # Final Summary for this pass
    elapsed = time.time() - start_time
    print(f"PASS {pass_num} COMPLETE")
    print(f"Successfully processed: {games_processed}/{len(game_ids_list)}")
    print(f"Failed: {games_failed}/{len(game_ids_list)}")
    print(f"Total time: {elapsed/60:.1f} minutes")
    
    # Check if we should continue to next pass
    if not failed_game_ids:
        print(f"\nALL GAMES PROCESSED SUCCESSFULLY! No need for more passes.")
        break
    elif pass_num < MAX_PASSES:
        print(f"\nWill attempt pass {pass_num + 1} with {len(failed_game_ids)} failed games...")
        time.sleep(5)  # Brief pause between passes
    else:
        print(f"\nReached maximum passes ({MAX_PASSES}). {len(failed_game_ids)} games still failed.")

print(f"ALL PASSES COMPLETE")


=== STARTING PASS 1 ===

Total games to process: 7807
First game ID: 0012000001
Last game ID: 2072100016
Failures: 1
Failures: 50
Failures: 100
Failures: 150
Failures: 200
Failures: 250
Progress: 1000/7807 games (747 success, 253 failed) - 120.6 min elapsed
Failures: 300
Failures: 350
Failures: 400
Failures: 450
Failures: 500
Progress: 2000/7807 games (1464 success, 536 failed) - 240.6 min elapsed
Failures: 550
Failures: 600
Failures: 650
Failures: 700
Failures: 750
Failures: 800
Progress: 3000/7807 games (2178 success, 822 failed) - 360.6 min elapsed
Failures: 850
Failures: 900
Failures: 950
Failures: 1000
Failures: 1050
Failures: 1100
Failures: 1150
Failures: 1200
Failures: 1250
Failures: 1300
Failures: 1350
Failures: 1400
Failures: 1450
Failures: 1500
Failures: 1550
Failures: 1600
Failures: 1650
Failures: 1700
Failures: 1750
Failures: 1800
Failures: 1850
Failures: 1900
Failures: 1950
Failures: 2000
Failures: 2050
Failures: 2100
Failures: 2150
Failures: 2200
Failures: 2250
Failures:

This next cell block is code to concatenate all the various pass files together. This could have been more elegant but I hard coded the amount of passes I had for each dataset

In [ ]:
# Concatenate all the csvs
import pandas as pd
import os

# Dictionary to store all combined dataframes
combined_data = {}

# First Half files (3 passes)
fh_files = [
    ('first_half_players', 3),
    ('first_half_teams', 3),
    ('first_half_starters_bench', 3)
]

for base_name, num_passes in fh_files:
    all_dfs = []
    
    for pass_num in range(1, num_passes + 1):
        if pass_num == 1:
            filename = f'{base_name}.csv'
        else:
            filename = f'{base_name}_pt{pass_num}.csv'
        
        if os.path.exists(filename):
            df = pd.read_csv(filename)
            all_dfs.append(df)
            print(f" Loaded {filename}: {len(df)} records")
        else:
            print(f"{filename} not found, skipping")
    
    if all_dfs:
        combined = pd.concat(all_dfs, ignore_index=True)
        combined_data[base_name] = combined
        print(f"Combined {base_name}: {len(combined)} total records\n")

# Complete Game files (5 passes)
cg_files = [
    ('complete_game_players', 5),
    ('complete_game_teams', 5),
    ('complete_game_starters_bench', 5)
]

for base_name, num_passes in cg_files:
    all_dfs = []
    
    for pass_num in range(1, num_passes + 1):
        if pass_num == 1:
            filename = f'{base_name}.csv'
        else:
            filename = f'{base_name}_pt{pass_num}.csv'
        
        if os.path.exists(filename):
            df = pd.read_csv(filename)
            all_dfs.append(df)
            print(f"Loaded {filename}: {len(df)} records")
        else:
            print(f"{filename} not found, skipping")
    
    if all_dfs:
        combined = pd.concat(all_dfs, ignore_index=True)
        combined_data[base_name] = combined
        print(f"Combined {base_name}: {len(combined)} total records\n")

# BoxScoreSummary files (9 passes)
bss_files = [
    ('game_summary', 9),
    ('team_stats', 9),
    ('refs', 9),
    ('inactive_players', 9),
    ('game_info', 9),
    ('points_by_quarter', 9),
    ('last_meeting', 9)
]

for base_name, num_passes in bss_files:
    all_dfs = []
    
    for pass_num in range(1, num_passes + 1):
        if pass_num == 1:
            filename = f'{base_name}.csv'
        else:
            filename = f'{base_name}_pt{pass_num}.csv'
        
        if os.path.exists(filename):
            df = pd.read_csv(filename)
            all_dfs.append(df)
            print(f"Loaded {filename}: {len(df)} records")
        else:
            print(f"{filename} not found, skipping")
    
    if all_dfs:
        combined = pd.concat(all_dfs, ignore_index=True)
        combined_data[base_name] = combined
        print(f"  📊 Combined {base_name}: {len(combined)} total records\n")

# Save all combined files
print("SAVING COMBINED FILES")

for name, df in combined_data.items():
    output_filename = f'{name}_COMBINED.csv'
    df.to_csv(output_filename, index=False)
    print(f"Saved {output_filename}: {len(df)} records")

print(f"\nAll files combined and saved!")
print(f"Total combined datasets: {len(combined_data)}")

=== CONCATENATING MULTI-PASS CSV FILES ===

Processing First Half files (3 passes)...
  ✅ Loaded first_half_players.csv: 120187 records
  ✅ Loaded first_half_players_pt2.csv: 22239 records
  ✅ Loaded first_half_players_pt3.csv: 5716 records
  📊 Combined first_half_players: 148142 total records

  ✅ Loaded first_half_teams.csv: 12656 records
  ✅ Loaded first_half_teams_pt2.csv: 2354 records
  ✅ Loaded first_half_teams_pt3.csv: 600 records
  📊 Combined first_half_teams: 15610 total records

  ✅ Loaded first_half_starters_bench.csv: 25312 records
  ✅ Loaded first_half_starters_bench_pt2.csv: 4708 records
  ✅ Loaded first_half_starters_bench_pt3.csv: 1200 records
  📊 Combined first_half_starters_bench: 31220 total records

Processing Complete Game files (5 passes)...
  ✅ Loaded complete_game_players.csv: 142397 records
  ✅ Loaded complete_game_players_pt2.csv: 43648 records
  ✅ Loaded complete_game_players_pt3.csv: 10429 records
  ✅ Loaded complete_game_players_pt4.csv: 1537 records
  ✅ Lo

This next cell block does a quick pass to check if we have a complete dataset.

In [ ]:
# Check for missing games in combined datasets
import pandas as pd
import os

# Load the original game IDs list
original_game_ids = pd.read_csv('unique_game_ids.csv', dtype={'GAME_ID': str})
total_games = len(original_game_ids)
all_game_ids = set(original_game_ids['GAME_ID'].tolist())

print(f"Total games expected: {total_games}")
print(f"First game ID: {original_game_ids['GAME_ID'].iloc[0]}")
print(f"Last game ID: {original_game_ids['GAME_ID'].iloc[-1]}")

# Dictionary to track coverage by dataset
coverage_report = {}

# Check First Half datasets
print("FIRST HALF DATASETS:")
fh_files = [
    'first_half_players_COMBINED.csv',
    'first_half_teams_COMBINED.csv',
    'first_half_starters_bench_COMBINED.csv'
]

for filename in fh_files:
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        if 'GAME_ID' in df.columns:
            unique_games = df['GAME_ID'].nunique()
            game_ids_in_file = set(df['GAME_ID'].dropna().astype(str).unique())
            missing_games = all_game_ids - game_ids_in_file
            coverage_pct = (unique_games / total_games) * 100
            
            coverage_report[filename] = {
                'games_found': unique_games,
                'missing_count': len(missing_games),
                'coverage_pct': coverage_pct,
                'missing_ids': sorted(missing_games)
            }
            
            print(f"  {filename}:")
            print(f"    Games found: {unique_games}/{total_games} ({coverage_pct:.1f}%)")
            print(f"    Missing: {len(missing_games)}")
        else:
            print(f"{filename}: No GAME_ID column found")
    else:
        print(f"{filename} not found")


# Check Complete Game datasets
print("COMPLETE GAME DATASETS:")
cg_files = [
    'complete_game_players_COMBINED.csv',
    'complete_game_teams_COMBINED.csv',
    'complete_game_starters_bench_COMBINED.csv'
]

for filename in cg_files:
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        if 'GAME_ID' in df.columns:
            unique_games = df['GAME_ID'].nunique()
            game_ids_in_file = set(df['GAME_ID'].dropna().astype(str).unique())
            missing_games = all_game_ids - game_ids_in_file
            coverage_pct = (unique_games / total_games) * 100
            
            coverage_report[filename] = {
                'games_found': unique_games,
                'missing_count': len(missing_games),
                'coverage_pct': coverage_pct,
                'missing_ids': sorted(missing_games)
            }
            
            print(f"{filename}:")
            print(f"Games found: {unique_games}/{total_games} ({coverage_pct:.1f}%)")
            print(f"Missing: {len(missing_games)}")
        else:
            print(f"{filename}: No GAME_ID column found")
    else:
        print(f"{filename} not found")

# Check BoxScoreSummary datasets
print("BOX SCORE SUMMARY DATASETS:")
bss_files = [
    'game_summary_COMBINED.csv',
    'team_stats_COMBINED.csv',
    'refs_COMBINED.csv',
    'inactive_players_COMBINED.csv',
    'game_info_COMBINED.csv',
    'points_by_quarter_COMBINED.csv',
    'last_meeting_COMBINED.csv'
]

for filename in bss_files:
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        if 'GAME_ID' in df.columns:
            unique_games = df['GAME_ID'].nunique()
            game_ids_in_file = set(df['GAME_ID'].dropna().astype(str).unique())
            missing_games = all_game_ids - game_ids_in_file
            coverage_pct = (unique_games / total_games) * 100
            
            coverage_report[filename] = {
                'games_found': unique_games,
                'missing_count': len(missing_games),
                'coverage_pct': coverage_pct,
                'missing_ids': sorted(missing_games)
            }
            
            print(f"{filename}:")
            print(f"Games found: {unique_games}/{total_games} ({coverage_pct:.1f}%)")
            print(f"Missing: {len(missing_games)}")
        else:
            print(f"{filename}: No GAME_ID column found")
    else:
        print(f"{filename} not found")

# Summary
print("SUMMARY:")
print(f"Total expected games: {total_games}")

if coverage_report:
    best_coverage = max(coverage_report.items(), key=lambda x: x[1]['coverage_pct'])
    worst_coverage = min(coverage_report.items(), key=lambda x: x[1]['coverage_pct'])
    
    print(f"\nBest coverage: {best_coverage[0]}")
    print(f"  {best_coverage[1]['games_found']}/{total_games} ({best_coverage[1]['coverage_pct']:.1f}%)")
    
    print(f"\nWorst coverage: {worst_coverage[0]}")
    print(f"  {worst_coverage[1]['games_found']}/{total_games} ({worst_coverage[1]['coverage_pct']:.1f}%)")
    
    # Find games missing from ALL datasets
    all_missing = set.intersection(*[set(v['missing_ids']) for v in coverage_report.values()])
    
    if all_missing:
        print(f"\nGames missing from ALL datasets: {len(all_missing)}")
        print(f"Sample missing IDs: {list(all_missing)[:10]}")
        
        # Save to CSV
        missing_df = pd.DataFrame({'GAME_ID': sorted(all_missing)})
        missing_df.to_csv('games_missing_from_all_datasets.csv', index=False)
        print(f" Saved to 'games_missing_from_all_datasets.csv'")
    else:
        print(f"\nNo games are missing from ALL datasets!")
        print("(Some datasets may have missing games, but coverage varies)")


=== CHECKING FOR MISSING GAMES ===

Total games expected: 7807
First game ID: 0012000001
Last game ID: 2072100016


FIRST HALF DATASETS:
  first_half_players_COMBINED.csv:
    Games found: 7805/7807 (100.0%)
    Missing: 5408
  first_half_teams_COMBINED.csv:
    Games found: 7805/7807 (100.0%)
    Missing: 5408
  first_half_starters_bench_COMBINED.csv:
    Games found: 7805/7807 (100.0%)
    Missing: 5408


COMPLETE GAME DATASETS:
  complete_game_players_COMBINED.csv:
    Games found: 7807/7807 (100.0%)
    Missing: 5407
  complete_game_teams_COMBINED.csv:
    Games found: 7807/7807 (100.0%)
    Missing: 5407
  complete_game_starters_bench_COMBINED.csv:
    Games found: 7807/7807 (100.0%)
    Missing: 5407


BOX SCORE SUMMARY DATASETS:
  game_summary_COMBINED.csv:
    Games found: 7801/7807 (99.9%)
    Missing: 5410
  team_stats_COMBINED.csv:
    Games found: 7803/7807 (99.9%)
    Missing: 7807
  refs_COMBINED.csv:
    Games found: 7807/7807 (100.0%)
    Missing: 7807
  inactive_player

Upon inspecting that data we had thoughts about pulling more specific player information but this was incredibly cumbersome with the API since we would have to make individual pulls on each PLAYER_ID. In thinking about it more it did not make sense to go through the trouble since we want the model to generalize well since we're using an old dataset and players move teams, players retire, and new players arrive.

The next cell block is for feature engineering. It is a bit of a mess and desperately needs to be refactored. We added more code snippets to add more features to the datasets. The need for more opponent features created another nested loop which felt easier at the time but simply kept growing it would have been better to replace it with a function. Also the dataset got to be over 1,000 columns and I added some redundant columns because I forgot which ones were already there.

In [ ]:
# Feature Engineering
import pandas as pd
import numpy as np

# Configuration
time_windows = [7, 14, 30] # 1 week, 2 week, and 1 month rolling snapshots
player_groups = [1, 2, 3, 4, 5, 6, 7, 'rest'] #top 7 of the rotation averages and lump sum for rest of team

# Stats to average
player_stats_avg = ['PTS', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA',
                    'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PLUS_MINUS']
player_stats_total = ['MIN']

# Team Complete Game
team_stats_avg_cg = ['PTS', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA',
                    'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PLUS_MINUS']
team_stats_total_cg = ['MIN']

# Team First Half
team_stats_avg_fh = ['PTS', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA',
                    'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PLUS_MINUS']
team_stats_total_fh = ['MIN']

# Load our data
cg_players = pd.read_csv('complete_game_players_COMBINED.csv')
cg_teams = pd.read_csv('complete_game_teams_COMBINED.csv')
fh_players = pd.read_csv('first_half_players_COMBINED.csv')
fh_teams = pd.read_csv('first_half_teams_COMBINED.csv')
games_master = pd.read_csv('nba_games_2021_to_2024.csv')
game_summary = pd.read_csv('game_summary_COMBINED.csv')
refs = pd.read_csv('refs_COMBINED.csv')
last_meeting = pd.read_csv('last_meeting_COMBINED.csv')

# Cast numeric for the above
for col in player_stats_avg + player_stats_total:
    if col in cg_players.columns:
        cg_players[col] = pd.to_numeric(cg_players[col], errors='coerce')
        fh_players[col] = pd.to_numeric(fh_players[col], errors='coerce')

for col in team_stats_avg + team_stats_total:
    if col in cg_teams.columns:
        cg_teams[col] = pd.to_numeric(cg_teams[col], errors='coerce')
        fh_teams[col] = pd.to_numeric(fh_teams[col], errors='coerce')

# Format Dates
games_master['GAME_DATE'] = pd.to_datetime(games_master['GAME_DATE'])
last_meeting['LAST_GAME_DATE_EST'] = pd.to_datetime(last_meeting['LAST_GAME_DATE_EST'], errors='coerce')


# Creating an index with unique game_id and date
game_date_map = games_master[['GAME_ID', 'GAME_DATE']].drop_duplicates('GAME_ID').set_index('GAME_ID')['GAME_DATE']

# Apply it
for df in [cg_players, fh_players, cg_teams, fh_teams]:
    df['GAME_DATE'] = df['GAME_ID'].map(game_date_map)

## Computing Second-Half Stats since we pulled first half and complete game
# Merge Key
cg_teams['merge_key'] = cg_teams['GAME_ID'].astype(str) + '_' + cg_teams['TEAM_ID'].astype(str)
fh_teams['merge_key'] = fh_teams['GAME_ID'].astype(str) + '_' + fh_teams['TEAM_ID'].astype(str)

# Merge Alignment
merged = cg_teams.merge(
    fh_teams[['merge_key', 'PTS', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'REB', 'AST', 'TO', 'STL', 'BLK']], 
    on='merge_key', 
    how='left',
    suffixes=('', '_fh')
)

# Second-half stats to compute
second_half_stats = ['PTS', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'REB', 'AST', 'TO', 'STL', 'BLK']

# Quick diff for counting stats
for stat in second_half_stats:
    cg_teams[f'second_half_{stat}'] = merged[stat] - merged[f'{stat}_fh']

# Calculate percentage stats
cg_teams['second_half_FG_PCT'] = np.where(
    cg_teams['second_half_FGA'] > 0,
    cg_teams['second_half_FGM'] / cg_teams['second_half_FGA'],
    np.nan
)
cg_teams['second_half_FG3_PCT'] = np.where(
    cg_teams['second_half_FG3A'] > 0,
    cg_teams['second_half_FG3M'] / cg_teams['second_half_FG3A'],
    np.nan
)
cg_teams['second_half_FT_PCT'] = np.where(
    cg_teams['second_half_FTA'] > 0,
    cg_teams['second_half_FTM'] / cg_teams['second_half_FTA'],
    np.nan
)

# Kill merge key since its unnessary now
cg_teams.drop('merge_key', axis=1, inplace=True)
fh_teams.drop('merge_key', axis=1, inplace=True)

# Add it to the stats list
team_stats_avg_cg.extend([f'second_half_{stat}' for stat in second_half_stats])
team_stats_avg_cg.extend(['second_half_FG_PCT', 'second_half_FG3_PCT', 'second_half_FT_PCT'])

## Players
# Lasso stats into time windows
def calculate_player_time_features(player_df, game_id, game_date, player_id, team_id, days):
    window_start = game_date - pd.Timedelta(days=days)
    
    player_games = player_df[
        (player_df['PLAYER_ID'] == player_id) &
        (player_df['TEAM_ID'] == team_id) &
        (player_df['GAME_DATE'] >= window_start) &
        (player_df['GAME_DATE'] < game_date) &
        (player_df['GAME_ID'] != game_id)
    ].copy()
    
    # Excludes inactivity
    if len(player_games) == 0:
        base_features = {stat: np.nan for stat in player_stats_total + player_stats_avg}
        base_features['FG_PCT'] = np.nan
        base_features['FG3_PCT'] = np.nan
        base_features['FT_PCT'] = np.nan
        return base_features
    
    features = {}
    
    # Sum counting stats for percentages later
    for stat in player_stats_total:
        features[stat] = player_games[stat].sum()
        
    # Averages for features
    for stat in player_stats_avg:
        features[stat] = player_games[stat].mean()
    
    # Calculate percentages from underlying counting stats instead of averaging percentages
    total_fgm = player_games['FGM'].sum()
    total_fga = player_games['FGA'].sum()
    total_fg3m = player_games['FG3M'].sum()
    total_fg3a = player_games['FG3A'].sum()
    total_ftm = player_games['FTM'].sum()
    total_fta = player_games['FTA'].sum()
    
    features['FG_PCT'] = total_fgm / total_fga if total_fga > 0 else np.nan
    features['FG3_PCT'] = total_fg3m / total_fg3a if total_fg3a > 0 else np.nan
    features['FT_PCT'] = total_ftm / total_fta if total_fta > 0 else np.nan
    
    return features

# Ranking our top players, which is done on a 30 day basis
# 30 day basis will show dropoffs of top players.
def get_team_top_players(player_df, game_id, game_date, team_id):
    window_start = game_date - pd.Timedelta(days=30)
    
    team_players = player_df[
        (player_df['TEAM_ID'] == team_id) &
        (player_df['GAME_DATE'] >= window_start) &
        (player_df['GAME_DATE'] < game_date) &
        (player_df['GAME_ID'] != game_id)
    ].copy()
    
    # Excludes inactivity
    if len(team_players) == 0:
        return []
    
    player_minutes = team_players.groupby('PLAYER_ID')['MIN'].sum().sort_values(ascending=False)
    return player_minutes.index.tolist()

## Teamwide
# Lasso stats into time windows
def calculate_team_time_features(team_df, game_id, game_date, team_id, days, stats_avg, stats_total):
    window_start = game_date - pd.Timedelta(days=days)
    
    team_games = team_df[
        (team_df['TEAM_ID'] == team_id) &
        (team_df['GAME_DATE'] >= window_start) &
        (team_df['GAME_DATE'] < game_date) &
        (team_df['GAME_ID'] != game_id)
    ].copy()
    
    if len(team_games) == 0:
        base_features = {stat: np.nan for stat in stats_total + stats_avg}  # Changed
        base_features['FG_PCT'] = np.nan
        base_features['FG3_PCT'] = np.nan
        base_features['FT_PCT'] = np.nan
        base_features['WIN_PCT'] = np.nan
        return base_features
    
    features = {}
    
    # Sum totals
    for stat in stats_total:  # Changed
        features[stat] = team_games[stat].sum()
    
    # Average the averages (including second-half stats for CG only)
    for stat in stats_avg:  # Changed
        features[stat] = team_games[stat].mean()
    
    # Calculate TRUE percentages from summed makes/attempts
    total_fgm = team_games['FGM'].sum()
    total_fga = team_games['FGA'].sum()
    total_fg3m = team_games['FG3M'].sum()
    total_fg3a = team_games['FG3A'].sum()
    total_ftm = team_games['FTM'].sum()
    total_fta = team_games['FTA'].sum()
    
    features['FG_PCT'] = total_fgm / total_fga if total_fga > 0 else np.nan
    features['FG3_PCT'] = total_fg3m / total_fg3a if total_fg3a > 0 else np.nan
    features['FT_PCT'] = total_ftm / total_fta if total_fta > 0 else np.nan
    
    # Win percentage
    team_game_ids = team_games['GAME_ID'].unique()
    team_game_results = games_master[
        (games_master['GAME_ID'].isin(team_game_ids)) &
        (games_master['TEAM_ID'] == team_id)
    ]
    if len(team_game_results) > 0:
        features['WIN_PCT'] = (team_game_results['WL'] == 'W').mean()
    else:
        features['WIN_PCT'] = np.nan
    
    return features

## Functions for Schedule
# Rest days, back-to-back games, and some general game density
def calculate_schedule_features(games_master, game_id, game_date, team_id):
    team_games = games_master[
        (games_master['TEAM_ID'] == team_id) &
        (games_master['GAME_DATE'] < game_date)
    ].sort_values('GAME_DATE')
    
    features = {}
    
    if len(team_games) == 0:
        features['days_since_last_game'] = np.nan
        features['is_back_to_back'] = 0
        features['games_in_last_3_days'] = 0
        features['games_in_last_5_days'] = 0
        features['games_in_last_7_days'] = 0
    else:
        # Here we compute last game and back to backs
        last_game_date = team_games['GAME_DATE'].iloc[-1]
        features['days_since_last_game'] = (game_date - last_game_date).days
        features['is_back_to_back'] = 1 if features['days_since_last_game'] == 1 else 0
        
        # Game density
        recent_games_3d = team_games[team_games['GAME_DATE'] >= game_date - pd.Timedelta(days=3)]
        recent_games_5d = team_games[team_games['GAME_DATE'] >= game_date - pd.Timedelta(days=5)]
        recent_games_7d = team_games[team_games['GAME_DATE'] >= game_date - pd.Timedelta(days=7)]
        
        features['games_in_last_3_days'] = len(recent_games_3d)
        features['games_in_last_5_days'] = len(recent_games_5d)
        features['games_in_last_7_days'] = len(recent_games_7d)
    
    return features

# Lets get home and road game lengths (streaks of home or away)
def calculate_home_road_context(games_master, game_id, game_date, team_id):
    team_games = games_master[
        (games_master['TEAM_ID'] == team_id) &
        (games_master['GAME_DATE'] < game_date)
    ].sort_values('GAME_DATE')
    
    features = {}
    
    # Determine if current game is home
    current_game = games_master[(games_master['GAME_ID'] == game_id) & (games_master['TEAM_ID'] == team_id)]
    if len(current_game) > 0:
        matchup = current_game['MATCHUP'].iloc[0]
        features['is_home_game'] = 1 if 'vs.' in str(matchup) else 0 # vs and @ determined home vs away
    else:
        features['is_home_game'] = np.nan
    
    if len(team_games) == 0:
        # First game of season - count as game #1
        features['home_stand_game_number'] = 1 if features['is_home_game'] == 1 else 0
        features['road_trip_game_number'] = 1 if features['is_home_game'] == 0 else 0
        return features
    
    # Count consecutive home or road games (BEFORE current game)
    consecutive_count = 0
    
    for _, game in team_games.iloc[::-1].iterrows():
        matchup = str(game['MATCHUP'])
        is_home = 1 if 'vs.' in matchup else 0
        
        # Check if this past game matches current game's location
        if features['is_home_game'] == is_home:
            consecutive_count += 1
        else:
            break  # Different arena, stop counting
    
    # Add 1 to include the CURRENT game
    features['home_stand_game_number'] = consecutive_count + 1 if features['is_home_game'] == 1 else 0
    features['road_trip_game_number'] = consecutive_count + 1 if features['is_home_game'] == 0 else 0
    
    return features

## Record Features
# Win and Loss streaks
def calculate_streak_features(games_master, game_id, game_date, team_id, lookback_games=[3, 5, 10]):
    team_games = games_master[
        (games_master['TEAM_ID'] == team_id) &
        (games_master['GAME_DATE'] < game_date)
    ].sort_values('GAME_DATE')
    
    features = {}
    
    if len(team_games) == 0:
        features['current_streak'] = 0
        for n in lookback_games:
            features[f'wins_last_{n}'] = np.nan
        features['home_win_pct_last_10'] = np.nan
        features['road_win_pct_last_10'] = np.nan
        return features
    
    # Current streak (positive = wins, negative = losses)
    recent_results = team_games['WL'].iloc[::-1].values
    current_result = recent_results[0] if len(recent_results) > 0 else None
    streak = 0
    
    for result in recent_results:
        if result == current_result:
            streak += 1
        else:
            break
    
    features['current_streak'] = streak if current_result == 'W' else -streak
    
    # Wins in last N games
    for n in lookback_games:
        last_n = team_games.tail(n)
        if len(last_n) > 0:
            features[f'wins_last_{n}'] = (last_n['WL'] == 'W').sum()
        else:
            features[f'wins_last_{n}'] = np.nan
    
    # Home/Road splits
    last_10 = team_games.tail(10)
    home_games = last_10[last_10['MATCHUP'].str.contains('vs.', na=False)]
    road_games = last_10[last_10['MATCHUP'].str.contains('@', na=False)]
    
    features['home_win_pct_last_10'] = (home_games['WL'] == 'W').mean() if len(home_games) > 0 else np.nan
    features['road_win_pct_last_10'] = (road_games['WL'] == 'W').mean() if len(road_games) > 0 else np.nan
    
    return features

# Head to head record calculations (Team matched against their Opponent)
def calculate_head_to_head_features(games_master, last_meeting, game_id, game_date, team_id):
    features = {}
    
    # Get opponent team_id for this game
    game_teams = games_master[games_master['GAME_ID'] == game_id]['TEAM_ID'].unique()
    opponent_id = [t for t in game_teams if t != team_id]
    
    if len(opponent_id) == 0:
        features['days_since_last_matchup'] = np.nan
        features['won_last_matchup'] = np.nan
        features['point_diff_last_matchup'] = np.nan
        features['season_record_vs_opponent'] = np.nan
        return features
    
    opponent_id = opponent_id[0]
    
    # Last meeting info
    last_mtg = last_meeting[last_meeting['GAME_ID'] == game_id]
    if len(last_mtg) > 0 and pd.notna(last_mtg['LAST_GAME_DATE_EST'].iloc[0]):
        last_game_date = last_mtg['LAST_GAME_DATE_EST'].iloc[0]
        features['days_since_last_matchup'] = (game_date - last_game_date).days
        
        # Determine who won
        home_pts = last_mtg['LAST_GAME_HOME_TEAM_POINTS'].iloc[0]
        visitor_pts = last_mtg['LAST_GAME_VISITOR_TEAM_POINTS'].iloc[0]
        home_team = last_mtg['LAST_GAME_HOME_TEAM_ID'].iloc[0]
        
        if home_team == team_id:
            features['won_last_matchup'] = 1 if home_pts > visitor_pts else 0
            features['point_diff_last_matchup'] = home_pts - visitor_pts
        else:
            features['won_last_matchup'] = 1 if visitor_pts > home_pts else 0
            features['point_diff_last_matchup'] = visitor_pts - home_pts
    else:
        features['days_since_last_matchup'] = np.nan
        features['won_last_matchup'] = np.nan
        features['point_diff_last_matchup'] = np.nan
    
    # Season record vs opponent
    season_games = games_master[
        (games_master['TEAM_ID'] == team_id) &
        (games_master['GAME_DATE'] < game_date)
    ]
    
    # Find games against this opponent
    opponent_game_ids = games_master[
        (games_master['TEAM_ID'] == opponent_id)
    ]['GAME_ID'].unique()
    
    matchup_games = season_games[season_games['GAME_ID'].isin(opponent_game_ids)]
    
    if len(matchup_games) > 0:
        features['season_record_vs_opponent'] = (matchup_games['WL'] == 'W').sum()
    else:
        features['season_record_vs_opponent'] = 0
    
    return features

## Referee Features -- basically a join
def get_referee_ids(refs, game_id):
    game_refs = refs[refs['GAME_ID'] == game_id].sort_values('JERSEY_NUM')
    
    ref_ids = [np.nan, np.nan, np.nan]
    for i, (_, ref) in enumerate(game_refs.iterrows()):
        if i < 3:
            ref_ids[i] = ref['OFFICIAL_ID']
    
    return {
        'ref_1_id': ref_ids[0],
        'ref_2_id': ref_ids[1],
        'ref_3_id': ref_ids[2]
    }

## Opponent features -- this is calculating features for the opponent
def get_opponent_id(games_master, game_id, team_id):
    """Get the opponent team ID for this game"""
    game_teams = games_master[games_master['GAME_ID'] == game_id]['TEAM_ID'].unique()
    opponent_ids = [t for t in game_teams if t != team_id]
    return opponent_ids[0] if len(opponent_ids) > 0 else None

# Unique filter to make sure we are not counting duplicates
unique_games = games_master[['GAME_ID', 'GAME_DATE', 'TEAM_ID']].drop_duplicates()
unique_games = unique_games.sort_values('GAME_DATE')

# Index Reset
unique_games = unique_games.reset_index(drop=True)

# Preparing to run it
all_features = []

# Looping over all games
for idx, row in unique_games.iterrows():    
    if idx % 500 == 0:
        print(f"Progress: {idx}/{len(unique_games)}")
    
    game_id = row['GAME_ID']
    game_date = row['GAME_DATE']
    team_id = row['TEAM_ID']
    
    game_features = {
        'GAME_ID': game_id,
        'GAME_DATE': game_date,
        'TEAM_ID': team_id
    }
    
    # Player features
    top_players_cg = get_team_top_players(cg_players, game_id, game_date, team_id)
    top_players_fh = get_team_top_players(fh_players, game_id, game_date, team_id)
    
    for days in time_windows:
        # Complete game players
        for n in player_groups:
            if n == 'rest':
                players = top_players_cg[7:] if len(top_players_cg) > 7 else []
            else:
                players = top_players_cg[:n] if len(top_players_cg) >= n else []
            
            if len(players) == 0:
                for stat in player_stats_total + player_stats_avg + ['FG_PCT', 'FG3_PCT', 'FT_PCT']:
                    game_features[f'top{n}_cg_{stat.lower()}_{days}d'] = np.nan
                continue
            
            player_features_list = [
                calculate_player_time_features(cg_players, game_id, game_date, p, team_id, days)
                for p in players
            ]
            
            for stat in player_stats_total + player_stats_avg + ['FG_PCT', 'FG3_PCT', 'FT_PCT']:
                values = [pf[stat] for pf in player_features_list if not pd.isna(pf[stat])]
                game_features[f'top{n}_cg_{stat.lower()}_{days}d'] = np.mean(values) if values else np.nan
        
        # First half players
        for n in player_groups:
            if n == 'rest':
                players = top_players_fh[7:] if len(top_players_fh) > 7 else []
            else:
                players = top_players_fh[:n] if len(top_players_fh) >= n else []
            
            if len(players) == 0:
                for stat in player_stats_total + player_stats_avg + ['FG_PCT', 'FG3_PCT', 'FT_PCT']:
                    game_features[f'top{n}_fh_{stat.lower()}_{days}d'] = np.nan
                continue
            
            player_features_list = [
                calculate_player_time_features(fh_players, game_id, game_date, p, team_id, days)
                for p in players
            ]
            
            for stat in player_stats_total + player_stats_avg + ['FG_PCT', 'FG3_PCT', 'FT_PCT']:
                values = [pf[stat] for pf in player_features_list if not pd.isna(pf[stat])]
                game_features[f'top{n}_fh_{stat.lower()}_{days}d'] = np.mean(values) if values else np.nan
    
    # Team features
    for days in time_windows:
    # Complete game with second-half stats
        team_cg = calculate_team_time_features(cg_teams, game_id, game_date, team_id, days, 
                                                team_stats_avg_cg, team_stats_total_cg)
        for stat, value in team_cg.items():
            game_features[f'team_cg_{stat.lower()}_{days}d'] = value
        
        # First half WITHOUT second-half stats
        team_fh = calculate_team_time_features(fh_teams, game_id, game_date, team_id, days,
                                                team_stats_avg_fh, team_stats_total_fh)
        for stat, value in team_fh.items():
            game_features[f'team_fh_{stat.lower()}_{days}d'] = value
    
    # Schedule and Record Featurres
    schedule_feats = calculate_schedule_features(games_master, game_id, game_date, team_id)
    game_features.update(schedule_feats)
    
    home_road_feats = calculate_home_road_context(games_master, game_id, game_date, team_id)
    game_features.update(home_road_feats)
    
    streak_feats = calculate_streak_features(games_master, game_id, game_date, team_id)
    game_features.update(streak_feats)
    
    h2h_feats = calculate_head_to_head_features(games_master, last_meeting, game_id, game_date, team_id)
    game_features.update(h2h_feats)
    
    # Ref features
    ref_feats = get_referee_ids(refs, game_id)
    game_features.update(ref_feats)
    
    # Current Game Stats columns
    current_game_fh = fh_teams[
        (fh_teams['GAME_ID'] == game_id) & 
        (fh_teams['TEAM_ID'] == team_id)
    ]
    
    if len(current_game_fh) > 0:
        game_features['current_fh_pts'] = current_game_fh['PTS'].iloc[0]
        game_features['current_fh_fgm'] = current_game_fh['FGM'].iloc[0]
        game_features['current_fh_fga'] = current_game_fh['FGA'].iloc[0]
        game_features['current_fh_fg3m'] = current_game_fh['FG3M'].iloc[0]
        game_features['current_fh_fg3a'] = current_game_fh['FG3A'].iloc[0]
        game_features['current_fh_ftm'] = current_game_fh['FTM'].iloc[0]
        game_features['current_fh_fta'] = current_game_fh['FTA'].iloc[0]
        game_features['current_fh_reb'] = current_game_fh['REB'].iloc[0]
        game_features['current_fh_ast'] = current_game_fh['AST'].iloc[0]
        game_features['current_fh_to'] = current_game_fh['TO'].iloc[0]
        game_features['current_fh_stl'] = current_game_fh['STL'].iloc[0]
        game_features['current_fh_blk'] = current_game_fh['BLK'].iloc[0]
        game_features['current_fh_pf'] = current_game_fh['PF'].iloc[0]
        
        # Calculate percentages and pace
        fga = current_game_fh['FGA'].iloc[0]
        fg3a = current_game_fh['FG3A'].iloc[0]
        fta = current_game_fh['FTA'].iloc[0]
        
        game_features['current_fh_fg_pct'] = current_game_fh['FGM'].iloc[0] / fga if fga > 0 else np.nan
        game_features['current_fh_fg3_pct'] = current_game_fh['FG3M'].iloc[0] / fg3a if fg3a > 0 else np.nan
        game_features['current_fh_ft_pct'] = current_game_fh['FTM'].iloc[0] / fta if fta > 0 else np.nan
        game_features['current_fh_pace'] = fga + current_game_fh['TO'].iloc[0]
    else:
        fh_stats = ['pts', 'fgm', 'fga', 'fg3m', 'fg3a', 'ftm', 'fta', 'reb', 'ast', 'to', 
                    'stl', 'blk', 'pf', 'fg_pct', 'fg3_pct', 'ft_pct', 'pace']
        for stat in fh_stats:
            game_features[f'current_fh_{stat}'] = np.nan

    ## Opponnet Features
    opponent_id = get_opponent_id(games_master, game_id, team_id)
    
    if opponent_id is not None:
        # Opponent team features for each time window
        for days in time_windows:
            opp_team_cg = calculate_team_time_features(cg_teams, game_id, game_date, opponent_id, days,
                                                        team_stats_avg_cg, team_stats_total_cg)
            for stat, value in opp_team_cg.items():
                game_features[f'opp_team_cg_{stat.lower()}_{days}d'] = value
            
            opp_team_fh = calculate_team_time_features(fh_teams, game_id, game_date, opponent_id, days,
                                                        team_stats_avg_fh, team_stats_total_fh)
            for stat, value in opp_team_fh.items():
                game_features[f'opp_team_fh_{stat.lower()}_{days}d'] = value
        
        # Opponent schedule features
        opp_schedule_feats = calculate_schedule_features(games_master, game_id, game_date, opponent_id)
        for key, value in opp_schedule_feats.items():
            game_features[f'opp_{key}'] = value
        
        # Opponent streak features
        opp_streak_feats = calculate_streak_features(games_master, game_id, game_date, opponent_id)
        for key, value in opp_streak_feats.items():
            game_features[f'opp_{key}'] = value
            
        # Team vs Opponent Differentials
        if not pd.isna(game_features.get('days_since_last_game')) and not pd.isna(game_features.get('opp_days_since_last_game')):
            game_features['rest_advantage'] = game_features['days_since_last_game'] - game_features['opp_days_since_last_game']
        else:
            game_features['rest_advantage'] = np.nan
        
        # Win percentage differential
        for days in time_windows:
            team_win_pct = game_features.get(f'team_cg_win_pct_{days}d', np.nan)
            opp_win_pct = game_features.get(f'opp_team_cg_win_pct_{days}d', np.nan)
            if not pd.isna(team_win_pct) and not pd.isna(opp_win_pct):
                game_features[f'win_pct_diff_{days}d'] = team_win_pct - opp_win_pct
            else:
                game_features[f'win_pct_diff_{days}d'] = np.nan
        
        # Recent form differential (wins in last 5)
        team_wins_5 = game_features.get('wins_last_5', np.nan)
        opp_wins_5 = game_features.get('opp_wins_last_5', np.nan)
        if not pd.isna(team_wins_5) and not pd.isna(opp_wins_5):
            game_features['recent_form_diff'] = team_wins_5 - opp_wins_5
        else:
            game_features['recent_form_diff'] = np.nan
        
        # Scoring differential
        for days in time_windows:
            team_pts = game_features.get(f'team_cg_pts_{days}d', np.nan)
            opp_pts = game_features.get(f'opp_team_cg_pts_{days}d', np.nan)
            if not pd.isna(team_pts) and not pd.isna(opp_pts):
                game_features[f'avg_scoring_diff_{days}d'] = team_pts - opp_pts
            else:
                game_features[f'avg_scoring_diff_{days}d'] = np.nan
        
        # Opponent Current Game Columns        
        opp_game_fh = fh_teams[
            (fh_teams['GAME_ID'] == game_id) & 
            (fh_teams['TEAM_ID'] == opponent_id)
        ]
        
        if len(opp_game_fh) > 0:
            game_features['opp_current_fh_pts'] = opp_game_fh['PTS'].iloc[0]
            game_features['opp_current_fh_fgm'] = opp_game_fh['FGM'].iloc[0]
            game_features['opp_current_fh_fga'] = opp_game_fh['FGA'].iloc[0]
            game_features['opp_current_fh_fg3m'] = opp_game_fh['FG3M'].iloc[0]
            game_features['opp_current_fh_fg3a'] = opp_game_fh['FG3A'].iloc[0]
            game_features['opp_current_fh_reb'] = opp_game_fh['REB'].iloc[0]
            game_features['opp_current_fh_ast'] = opp_game_fh['AST'].iloc[0]
            game_features['opp_current_fh_to'] = opp_game_fh['TO'].iloc[0]
            game_features['opp_current_fh_pf'] = opp_game_fh['PF'].iloc[0]
            
            opp_fga = opp_game_fh['FGA'].iloc[0]
            opp_fg3a = opp_game_fh['FG3A'].iloc[0]
            
            game_features['opp_current_fh_fg_pct'] = opp_game_fh['FGM'].iloc[0] / opp_fga if opp_fga > 0 else np.nan
            game_features['opp_current_fh_fg3_pct'] = opp_game_fh['FG3M'].iloc[0] / opp_fg3a if opp_fg3a > 0 else np.nan
            game_features['opp_current_fh_pace'] = opp_fga + opp_game_fh['TO'].iloc[0]
        else:
            opp_fh_stats = ['pts', 'fgm', 'fga', 'fg3m', 'fg3a', 'reb', 'ast', 'to', 'pf', 
                            'fg_pct', 'fg3_pct', 'pace']
            for stat in opp_fh_stats:
                game_features[f'opp_current_fh_{stat}'] = np.nan
        
        # Halftime Stats        
        if len(current_game_fh) > 0 and len(opp_game_fh) > 0:
            
            # Totals
            game_features['halftime_total'] = current_game_fh['PTS'].iloc[0] + opp_game_fh['PTS'].iloc[0]
            
            # PAce
            team_pace = game_features.get('current_fh_pace', 0)
            opp_pace = game_features.get('opp_current_fh_pace', 0)
            game_features['halftime_total_pace'] = team_pace + opp_pace
            
            # Shooting PCTs
            team_fg_pct = game_features.get('current_fh_fg_pct', np.nan)
            opp_fg_pct = game_features.get('opp_current_fh_fg_pct', np.nan)
            if not pd.isna(team_fg_pct) and not pd.isna(opp_fg_pct):
                game_features['halftime_combined_fg_pct'] = (team_fg_pct + opp_fg_pct) / 2
            else:
                game_features['halftime_combined_fg_pct'] = np.nan
            
            # Turnovers
            game_features['halftime_total_to'] = current_game_fh['TO'].iloc[0] + opp_game_fh['TO'].iloc[0]
            
            # Scoring
            team_avg = game_features.get('team_fh_pts_7d', np.nan)
            opp_avg = game_features.get('opp_team_fh_pts_7d', np.nan)
            team_current = current_game_fh['PTS'].iloc[0]
            opp_current = opp_game_fh['PTS'].iloc[0]
            
            if not pd.isna(team_avg) and not pd.isna(opp_avg):
                team_var = team_current - team_avg
                opp_var = opp_current - opp_avg
                game_features['halftime_combined_scoring_variance'] = team_var + opp_var
            else:
                game_features['halftime_combined_scoring_variance'] = np.nan
            
            # Lead
            game_features['halftime_lead_abs'] = abs(current_game_fh['PTS'].iloc[0] - opp_game_fh['PTS'].iloc[0])
        
        else:
            game_features['halftime_total'] = np.nan
            game_features['halftime_total_pace'] = np.nan
            game_features['halftime_combined_fg_pct'] = np.nan
            game_features['halftime_total_to'] = np.nan
            game_features['halftime_combined_scoring_variance'] = np.nan
            game_features['halftime_lead_abs'] = np.nan
    
    ## Getting Second Half Targets
    current_game_cg = cg_teams[
        (cg_teams['GAME_ID'] == game_id) & 
        (cg_teams['TEAM_ID'] == team_id)
    ]
    
    # Team's second-half score
    if len(current_game_cg) > 0 and len(current_game_fh) > 0:
        complete_pts = current_game_cg['PTS'].iloc[0]
        first_half_pts = current_game_fh['PTS'].iloc[0]
        game_features['actual_second_half_pts'] = complete_pts - first_half_pts
    else:
        game_features['actual_second_half_pts'] = np.nan
    
    # Opponent's second-half score
    if opponent_id is not None:
        opp_game_cg = cg_teams[
            (cg_teams['GAME_ID'] == game_id) & 
            (cg_teams['TEAM_ID'] == opponent_id)
        ]
        
        if len(opp_game_cg) > 0 and len(opp_game_fh) > 0:
            opp_complete_pts = opp_game_cg['PTS'].iloc[0]
            opp_first_half_pts = opp_game_fh['PTS'].iloc[0]
            opp_second_half_pts = opp_complete_pts - opp_first_half_pts
            
            # Both teams combined for second half (this is the target)
            if not pd.isna(game_features['actual_second_half_pts']) and not pd.isna(opp_second_half_pts):
                game_features['actual_second_half_total'] = game_features['actual_second_half_pts'] + opp_second_half_pts
            else:
                game_features['actual_second_half_total'] = np.nan
        else:
            game_features['actual_second_half_total'] = np.nan
    else:
        game_features['actual_second_half_total'] = np.nan

    all_features.append(game_features)
    
# Create dataframe
features_df = pd.DataFrame(all_features)


Progress: 0/15613
Progress: 500/15613
Progress: 1000/15613
Progress: 1500/15613
Progress: 2000/15613
Progress: 2500/15613
Progress: 3000/15613
Progress: 3500/15613
Progress: 4000/15613
Progress: 4500/15613
Progress: 5000/15613
Progress: 5500/15613
Progress: 6000/15613
Progress: 6500/15613
Progress: 7000/15613
Progress: 7500/15613
Progress: 8000/15613
Progress: 8500/15613
Progress: 9000/15613
Progress: 9500/15613
Progress: 10000/15613
Progress: 10500/15613
Progress: 11000/15613
Progress: 11500/15613
Progress: 12000/15613
Progress: 12500/15613
Progress: 13000/15613
Progress: 13500/15613
Progress: 14000/15613
Progress: 14500/15613
Progress: 15000/15613
Progress: 15500/15613


Used this to inspect the dataset and then also write the dataset. They're split as a relic of testing.

In [36]:
features_df.head()
features_df.to_csv('nba_time_based_features.csv', index=False)

Next cell is using a mapping table to try and create a key based on the team and the date so we can join it to our NBA dataset. There were a number of games missing and a big bulk of those is because we only have half of the 2023 season.

In [ ]:
# Mapping table for betting line dataset to be joined to NBA data

import pandas as pd
import numpy as np

# Load data
print("Loading data...")
betting_data = pd.read_csv('betting_data.csv')
games_master = pd.read_csv('nba_games_2021_to_2024.csv')

# Format dates
betting_data['date'] = pd.to_datetime(betting_data['date'])
games_master['GAME_DATE'] = pd.to_datetime(games_master['GAME_DATE'])

# Create team abbreviation mapping
team_map = {
    'atl': 'ATL', 'bos': 'BOS', 'bkn': 'BKN', 'cha': 'CHA', 'chi': 'CHI',
    'cle': 'CLE', 'dal': 'DAL', 'den': 'DEN', 'det': 'DET', 'gs': 'GSW',
    'hou': 'HOU', 'ind': 'IND', 'lac': 'LAC', 'lal': 'LAL', 'mem': 'MEM',
    'mia': 'MIA', 'mil': 'MIL', 'min': 'MIN', 'no': 'NOP', 'nyk': 'NYK',
    'okc': 'OKC', 'orl': 'ORL', 'phi': 'PHI', 'phx': 'PHX', 'por': 'POR',
    'sac': 'SAC', 'sa': 'SAS', 'tor': 'TOR', 'utah': 'UTA', 'wsh': 'WAS',
    'nj': 'BKN', 'ny': 'NYK'
}

# Apply mapping
betting_data['away_team'] = betting_data['away'].map(team_map)
betting_data['home_team'] = betting_data['home'].map(team_map)

# Use dates and teams to join
betting_data['match_key'] = (
    betting_data['date'].dt.strftime('%Y-%m-%d') + '_' + 
    betting_data['away_team'] + '_' + 
    betting_data['home_team']
)

# Create a key for NBA games
games_master['is_home'] = games_master['MATCHUP'].str.contains('vs.', na=False)

# Separate home and away games
home_games = games_master[games_master['is_home'] == True][
    ['GAME_ID', 'GAME_DATE', 'TEAM_ABBREVIATION']
].copy()
away_games = games_master[games_master['is_home'] == False][
    ['GAME_ID', 'GAME_DATE', 'TEAM_ABBREVIATION']
].copy()

home_games.columns = ['GAME_ID', 'GAME_DATE', 'home_team']
away_games.columns = ['GAME_ID', 'GAME_DATE', 'away_team']

# Need a version for home and away
game_matchups = home_games.merge(away_games, on=['GAME_ID', 'GAME_DATE'])

# Create key for NBA games
game_matchups['match_key'] = (
    game_matchups['GAME_DATE'].dt.strftime('%Y-%m-%d') + '_' + 
    game_matchups['away_team'] + '_' + 
    game_matchups['home_team']
)

# Join based on keys
game_id_target = game_matchups.merge(
    betting_data[['match_key', 'h2_total']],
    on='match_key',
    how='left'
)

# Keep only GAME_ID and h2_total
game_id_target = game_id_target[['GAME_ID', 'h2_total']].drop_duplicates()

print(f"\nCreated GAME_ID to h2_total mapping")
print(f"Total games: {len(game_id_target)}")
print(f"Games with h2_total: {game_id_target['h2_total'].notna().sum()}")
print(f"Games missing h2_total: {game_id_target['h2_total'].isna().sum()}")

# Save the mapping
game_id_target.to_csv('game_id_to_h2_total.csv', index=False)
print(f"\nSaved to game_id_to_h2_total.csv")


Loading data...

✅ Created GAME_ID to h2_total mapping
Total games: 7804
Games with h2_total: 3158
Games missing h2_total: 4646

💾 Saved to game_id_to_h2_total.csv

Sample of mapping:
     GAME_ID  h2_total
0   42000406     112.5
1   42000405     112.5
2   42000404     113.5
3   42000403     113.0
4   42000402     112.0
5   42000401     112.5
6   42000306     111.5
7   42000305     105.5
8   42000316     109.5
9   42000304     108.0
10  42000315     109.0
11  42000303     113.5
12  42000314     109.5
13  42000302     110.5
14  42000313     111.5
15  42000301     111.0
16  42000312     112.0
17  42000207     106.0
18  42000311     112.0
19  42000217     109.0


Cell below actually does the join. Upon further investigation we realized that the failed joins were preseason and international exhibition games so we can drop them.

In [10]:
# Merge Betting Lines On and Filter
import pandas as pd
import numpy as np

features_df = pd.read_csv('nba_time_based_features.csv')
game_id_target = pd.read_csv('game_id_to_h2_total.csv')

# Join h2_total to features
features_with_target = features_df.merge(
    game_id_target,
    on='GAME_ID',
    how='left'
)

print(f"After join: {features_with_target.shape}")
print(f"Rows with h2_total: {features_with_target['h2_total'].notna().sum()}")
print(f"Rows without h2_total: {features_with_target['h2_total'].isna().sum()}")

# Drop rows without h2_total (preseason/exhibition games)
features_with_target = features_with_target[features_with_target['h2_total'].notna()]

print(f"\nAfter dropping preseason/exhibition: {features_with_target.shape}")

# Save the joined dataset
print(f"\nSaving to nba_features_with_target.csv...")
features_with_target.to_csv('nba_features_with_target.csv', index=False)
print(f"Saved!")


After join: (15613, 1383)
Rows with h2_total: 6316
Rows without h2_total: 9297

After dropping preseason/exhibition: (6316, 1383)

Saving to nba_features_with_target.csv...
Saved!


This next cell block preps the data for modelling. First it drops all columns that would cost data leaks (basically ones that have second half information) since we are using walk forward building dates are fine. We are testing on the second most recent season we had betting information on and before that is train data. We held out the last partial season of data for seperate validation processes.

In [11]:
# Train / Test splits for data

import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('nba_features_with_target.csv')
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

# Drop leakage columns
drop_columns = [
    # Rolling second-half team stats
    'team_cg_second_half_pts_7d', 'team_cg_second_half_fgm_7d', 
    'team_cg_second_half_fga_7d', 'team_cg_second_half_fg3m_7d',
    'team_cg_second_half_fg3a_7d', 'team_cg_second_half_ftm_7d',
    'team_cg_second_half_fta_7d', 'team_cg_second_half_reb_7d',
    'team_cg_second_half_ast_7d', 'team_cg_second_half_to_7d',
    'team_cg_second_half_stl_7d', 'team_cg_second_half_blk_7d',
    'team_cg_second_half_fg_pct_7d', 'team_cg_second_half_fg3_pct_7d',
    'team_cg_second_half_ft_pct_7d', 'team_cg_second_half_pts_14d',
    'team_cg_second_half_fgm_14d', 'team_cg_second_half_fga_14d',
    'team_cg_second_half_fg3m_14d', 'team_cg_second_half_fg3a_14d',
    'team_cg_second_half_ftm_14d', 'team_cg_second_half_fta_14d',
    'team_cg_second_half_reb_14d', 'team_cg_second_half_ast_14d',
    'team_cg_second_half_to_14d', 'team_cg_second_half_stl_14d',
    'team_cg_second_half_blk_14d', 'team_cg_second_half_fg_pct_14d',
    'team_cg_second_half_fg3_pct_14d', 'team_cg_second_half_ft_pct_14d',
    'team_cg_second_half_pts_30d', 'team_cg_second_half_fgm_30d',
    'team_cg_second_half_fga_30d', 'team_cg_second_half_fg3m_30d',
    'team_cg_second_half_fg3a_30d', 'team_cg_second_half_ftm_30d',
    'team_cg_second_half_fta_30d', 'team_cg_second_half_reb_30d',
    'team_cg_second_half_ast_30d', 'team_cg_second_half_to_30d',
    'team_cg_second_half_stl_30d', 'team_cg_second_half_blk_30d',
    'team_cg_second_half_fg_pct_30d', 'team_cg_second_half_fg3_pct_30d',
    'team_cg_second_half_ft_pct_30d',

    # Direct post-game leakage columns
    'actual_second_half_pts', 'actual_second_half_fgm',
    'actual_second_half_fga', 'actual_second_half_fg3m',
    'actual_second_half_fg3a', 'actual_second_half_ftm',
    'actual_second_half_fta', 'actual_second_half_reb',
    'actual_second_half_ast', 'actual_second_half_to',
    'actual_second_half_stl', 'actual_second_half_blk',
    'actual_second_half_fg_pct', 'actual_second_half_fg3_pct',
    'actual_second_half_ft_pct', 'actual_second_half_plus_minus'
]

df = df.drop(columns=drop_columns, errors='ignore')

# Get seasons
def get_nba_season(date):
    year = date.year
    month = date.month
    return f"{year}-{year+1}" if month >= 10 else f"{year-1}-{year}"

df['season'] = df['GAME_DATE'].apply(get_nba_season)
df = df.sort_values('GAME_DATE')

seasons = sorted(df['season'].unique())

# Train on earliest seasons, test on second-to-last, holdout on last (where we have partial data to validate seperately)
holdout_season = seasons[-1]
test_season = seasons[-2]
train_seasons = seasons[:-2]

# Filter data (need both target and betting line)
train_data = df[
    (df['season'].isin(train_seasons)) & 
    (df['actual_second_half_total'].notna()) &
    (df['h2_total'].notna())
]

test_data = df[
    (df['season'] == test_season) & 
    (df['actual_second_half_total'].notna()) &
    (df['h2_total'].notna())
]

holdout_data = df[
    (df['season'] == holdout_season) & 
    (df['actual_second_half_total'].notna())
]

# Print statements to give us a sense of dataset size
print(f"\nTrain: {len(train_data)} games")
print(f"Test: {len(test_data)} games")
print(f"Holdout: {len(holdout_data)} games")

# Prepare features (exclude metadata, target, and betting line)
metadata_cols = ['GAME_ID', 'GAME_DATE', 'TEAM_ID', 'season']
target_col = 'actual_second_half_total'
betting_line_col = 'h2_total'

feature_cols = [c for c in df.columns 
                if c not in metadata_cols + [target_col, betting_line_col]]

# Save datasets
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)
holdout_data.to_csv('holdout_data.csv', index=False)

with open('feature_list.txt', 'w') as f:
    for col in feature_cols:
        f.write(f"{col}\n")

print("\nSaved: train_data.csv, test_data.csv, holdout_data.csv, feature_list.txt")



Train: 2342 games
Test: 2646 games
Holdout: 1328 games

Saved: train_data.csv, test_data.csv, holdout_data.csv, feature_list.txt


In [52]:
# First XGBoost Model

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Load Data
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

# Format Dates
train_data['GAME_DATE'] = pd.to_datetime(train_data['GAME_DATE'])
test_data['GAME_DATE'] = pd.to_datetime(test_data['GAME_DATE'])

## Sorting Columns
# Filters
metadata_cols = ['GAME_ID', 'GAME_DATE', 'TEAM_ID', 'season']

# Target
target_col = 'actual_second_half_total'

# Comparison at end not used in modelling
betting_line_col = 'h2_total'

# Feature columns
drop_from_features = set(metadata_cols + [target_col])
if betting_line_col in train_data.columns:
    drop_from_features.add(betting_line_col)

feature_cols = [c for c in train_data.columns if c not in drop_from_features]

# Split into X and y
X_train = train_data[feature_cols]
y_train = train_data[target_col]

X_test = test_data[feature_cols]
y_test = test_data[target_col]

# Imputer using Median

missing_train = X_train.isnull().sum()
cols_with_missing = missing_train[missing_train > 0]

if len(cols_with_missing) > 0:
    for col in cols_with_missing.index:
        median_val = X_train[col].median()
        X_train[col].fillna(median_val, inplace=True)
        X_test[col].fillna(median_val, inplace=True)  # use train medians for test
    print("Missing values handled!")
else:
    print("No missing values!")

## Model

model = xgb.XGBRegressor(
    n_estimators=1800,
    learning_rate=0.018,
    max_depth=8,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.75,
    gamma=0.05,
    reg_alpha=0.2,
    reg_lambda=0.6,
    random_state=42,
    n_jobs=-1,
    early_stopping_rounds=120,
    eval_metric='rmse'
)

model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=50
)


# Evalutation 
y_pred_test = model.predict(X_test)

mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2_test = r2_score(y_test, y_pred_test)

print(f"\nTest Metrics:")
print(f"MAE:  {mae_test:.3f} points")
print(f"RMSE: {rmse_test:.3f} points")
print(f"R²:   {r2_test:.3f}")

# Export Feature Importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

# Store Predictions
test_results = test_data.copy()
test_results['actual_second_half_total_predicted'] = y_pred_test
test_results['prediction_error'] = np.abs(y_test - y_pred_test)

show_cols = ['GAME_ID', 'GAME_DATE', 'actual_second_half_total',
            'actual_second_half_total_predicted', 'prediction_error']
if betting_line_col in test_results.columns:
    test_results['edge_vs_line'] = test_results['actual_second_half_total_predicted'] - test_results[betting_line_col]
    show_cols.append(betting_line_col)
    show_cols.append('edge_vs_line')


# Save model and results
model.save_model('xgboost_model.json')
print("Saved xgboost_model.json")

feature_importance.to_csv('feature_importance.csv', index=False)
print("Saved feature_importance.csv")

test_results.to_csv('test_predictions.csv', index=False)
print("Saved test_predictions.csv")


Missing values handled!
[0]	validation_0-rmse:13.96279	validation_1-rmse:13.56825
[50]	validation_0-rmse:10.18589	validation_1-rmse:13.46736
[100]	validation_0-rmse:7.69453	validation_1-rmse:13.48778
[150]	validation_0-rmse:6.06479	validation_1-rmse:13.51003
[161]	validation_0-rmse:5.74957	validation_1-rmse:13.51027

Test Metrics:
MAE:  10.610 points
RMSE: 13.458 points
R²:   0.004
Saved xgboost_model.json
Saved feature_importance.csv
Saved test_predictions.csv


In [53]:
# XGboost using h2_total residual

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")

# Load Data
train_data = pd.read_csv("train_data.csv")
test_data = pd.read_csv("test_data.csv")

# Format dates
train_data["GAME_DATE"] = pd.to_datetime(train_data["GAME_DATE"])
test_data["GAME_DATE"] = pd.to_datetime(test_data["GAME_DATE"])

## Sorting Columns
# Filters
metadata_cols = ["GAME_ID", "GAME_DATE", "TEAM_ID", "season"]

# Target
target_col = "actual_second_half_total"

# Residual
vegas_col = "h2_total"

# Feature columns
feature_cols = [c for c in train_data.columns if c not in metadata_cols + [target_col]]

X_train = train_data[feature_cols].copy()
# Add Residual Here
y_train = train_data[target_col] - train_data[vegas_col]


X_test = test_data[feature_cols].copy()
# Add Residual Here
y_test = test_data[target_col] - test_data[vegas_col]

# Imputer for missing values uses median

for col in X_train.columns:
    if X_train[col].isnull().any():
        median_val = X_train[col].median()
        X_train[col].fillna(median_val, inplace=True)
        X_test[col].fillna(median_val, inplace=True)

## Model
model = xgb.XGBRegressor(
    n_estimators=1800,
    learning_rate=0.018,
    max_depth=8,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.75,
    gamma=0.05,
    reg_alpha=0.2,
    reg_lambda=0.6,
    random_state=42,
    n_jobs=-1,
    early_stopping_rounds=120,
    eval_metric='rmse'
)

model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=50
)

# Evalutation

y_pred_resid = model.predict(X_test)
mae_resid = mean_absolute_error(y_test, y_pred_resid)
rmse_resid = np.sqrt(mean_squared_error(y_test, y_pred_resid))
r2_resid = r2_score(y_test, y_pred_resid)

print(f"\nResidual Model Metrics:")
print(f"MAE:  {mae_resid:.3f} pts")
print(f"RMSE: {rmse_resid:.3f} pts")
print(f"R²:   {r2_resid:.3f}")

# Reconstruct Predictions
y_pred_total = y_pred_resid + X_test[vegas_col]
y_true_total = y_test + X_test[vegas_col]

mae_total = mean_absolute_error(y_true_total, y_pred_total)
rmse_total = np.sqrt(mean_squared_error(y_true_total, y_pred_total))
r2_total = r2_score(y_true_total, y_pred_total)

print("FULL TOTAL PERFORMANCE (Actual Second Half Totals)")
print(f"MAE:  {mae_total:.3f} pts")
print(f"RMSE: {rmse_total:.3f} pts")
print(f"R²:   {r2_total:.3f}")

# Prepping test results
test_results = test_data.copy()
test_results["predicted_residual"] = y_pred_resid
test_results["predicted_total"] = y_pred_total
test_results["residual_error"] = y_test - y_pred_resid
test_results["actual_residual"] = y_test
test_results["edge_vs_vegas"] = y_pred_resid

# Save everything
model.save_model("xgboost_residual_model.json")
test_results.to_csv("test_predictions_residual.csv", index=False)

print("Saved model (xgboost_residual_model.json)")
print("Saved predictions (test_predictions_residual.csv)")


[0]	validation_0-rmse:13.24557	validation_1-rmse:12.61378
[50]	validation_0-rmse:10.74339	validation_1-rmse:12.61934
[100]	validation_0-rmse:8.57413	validation_1-rmse:12.66605
[150]	validation_0-rmse:6.78661	validation_1-rmse:12.72903

Residual Model Metrics:
MAE:  9.735 pts
RMSE: 12.601 pts
R²:   0.002
FULL TOTAL PERFORMANCE (Actual Second Half Totals)
MAE:  9.735 pts
RMSE: 12.601 pts
R²:   0.127
Saved model (xgboost_residual_model.json)
Saved predictions (test_predictions_residual.csv)


In [54]:
# Model Comparison

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load models
resid = pd.read_csv("test_predictions.csv")                  # main (no residual) model
no_resid = pd.read_csv("test_predictions_residual.csv")      # residual model

# I named the columns badly...
def get_pred_col(df):
    if "predicted_total" in df.columns:
        return "predicted_total"
    elif "actual_second_half_total_predicted" in df.columns:
        return "actual_second_half_total_predicted"
    else:
        raise KeyError("still can't get the columns right")

pred_col_resid = get_pred_col(resid)
pred_col_no_resid = get_pred_col(no_resid)

# Targets
y_true = resid["actual_second_half_total"]
y_vegas = resid["h2_total"]

# Predictions
y_pred_no_residual = resid[pred_col_resid]
y_pred_residual = no_resid[pred_col_no_resid]

# Evaluator function
def evaluate(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

# Metrics Calc
results = {
    "Vegas Line": evaluate(y_true, y_vegas),
    "No Residual Model": evaluate(y_true, y_pred_no_residual),
    "Residual Model": evaluate(y_true, y_pred_residual),
}

# Summary output
summary = pd.DataFrame(results, index=["MAE", "RMSE", "R²"]).T
summary["Δ MAE vs Vegas"] = summary["MAE"] - summary.loc["Vegas Line", "MAE"]
summary["Δ RMSE vs Vegas"] = summary["RMSE"] - summary.loc["Vegas Line", "RMSE"]

print("SECOND HALF TOTALS – MODEL BENCHMARK COMPARISON")
print(summary.to_string(float_format=lambda x: f"{x:0.3f}"))


SECOND HALF TOTALS – MODEL BENCHMARK COMPARISON
                     MAE   RMSE    R²  Δ MAE vs Vegas  Δ RMSE vs Vegas
Vegas Line         9.738 12.618 0.124           0.000            0.000
No Residual Model 10.610 13.458 0.004           0.872            0.840
Residual Model     9.735 12.601 0.127          -0.004           -0.017


Basically as we were testing we found out that the Vegas line is actually very very good. Even without using first half data its actually better then our tuned XGBoost model. Knowing that we have the Vegas second half total we used that as a residual in another XGBoost model, which gave us some improvements over the Vegas line. Is this enough to make betting model that is profitable? We do not quite know yet. 

In [4]:
# Unsupervised learning

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import joblib
import warnings
warnings.filterwarnings("ignore")

# Loading training data
df = pd.read_csv("train_data.csv")
df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"])

## Sorting Columns
# Filters
metadata_cols = ["GAME_ID", "GAME_DATE", "TEAM_ID", "season"]

# Target
target_col = "actual_second_half_total"

# Comparison col
betting_col = "h2_total"

# Drop unnecceasry columns and keep only features
drop_cols = set(metadata_cols + [target_col, betting_col])
feature_cols = [c for c in df.columns if c not in drop_cols]

X = df[feature_cols].copy()

# Imputer with medium
missing = X.isna().sum().sum()
if missing > 0:
    for c in feature_cols:
        if X[c].isna().any():
            X[c].fillna(X[c].median(), inplace=True)
else:
    print("No imputing neccesary")


# Scale and apply PCA
scaler = StandardScaler(with_mean=True, with_std=True)
X_scaled = scaler.fit_transform(X)

# keep up to 30 PCs or fewer if features < 30
n_components = min(30, X_scaled.shape[1])
pca = PCA(n_components=n_components, svd_solver="auto", random_state=42)
X_pcs = pca.fit_transform(X_scaled)

explained = pca.explained_variance_ratio_.cumsum()
var_99 = np.argmax(explained >= 0.99) + 1 if (explained >= 0.99).any() else n_components

print(f"\nPCA components: {n_components}")
print(f"Explained variance (first 5 PCs cumulative): {explained[:5]}")
print(f"Components to reach ~99% variance: {var_99}")

# KMeans test and evaluating based on silhouette
best = {"k": None, "score": -1, "model": None, "labels": None}
for k in [6, 8, 10, 12]:
    km = KMeans(n_clusters=k, n_init=20, max_iter=500, random_state=42)
    labels = km.fit_predict(X_pcs)
    score = silhouette_score(X_pcs, labels) if len(np.unique(labels)) > 1 else -1
    print(f"k={k:2d} -> silhouette: {score:.4f}")
    if score > best["score"]:
        best = {"k": k, "score": score, "model": km, "labels": labels}

k_best = best["k"]
kmeans = best["model"]
labels = best["labels"]
print(f"\nSelected k={k_best} (silhouette={best['score']:.4f})")

# Display Cluster Information
pc_cols = [f"PC{i+1}" for i in range(X_pcs.shape[1])]
out = df[["GAME_ID", "GAME_DATE", "TEAM_ID", "season"]].copy()
for i, col in enumerate(pc_cols):
    out[col] = X_pcs[:, i]
out["cluster_id"] = labels

# Adding target col and betting col to explore clusters
if target_col in df.columns:
    out[target_col] = df[target_col].values
if betting_col in df.columns:
    out[betting_col] = df[betting_col].values
    if target_col in df.columns:
        out["edge_vs_line"] = out[target_col] - out[betting_col]

out.to_csv("pca_kmeans_train.csv", index=False)

# Summary for Cluster
if target_col in out.columns:
    agg_cols[target_col] = ["mean", "median"]
if betting_col in out.columns:
    agg_cols[betting_col] = ["mean", "median"]
if "edge_vs_line" in out.columns:
    agg_cols["edge_vs_line"] = ["mean", "median"]

summary = (
    out.groupby("cluster_id")
        .agg(**{k: pd.NamedAgg(column=k, aggfunc=v) for k, v in {
           **({target_col: "mean"} if target_col in out.columns else {}),
           **({betting_col: "mean"} if betting_col in out.columns else {}),
           **({"edge_vs_line": "mean"} if "edge_vs_line" in out.columns else {})
        }.items()})
        .rename(columns={
            target_col: "avg_actual_2H_total",
            betting_col: "avg_h2_total",
            "edge_vs_line": "avg_edge_vs_line"
        })
        .reset_index()
)
sizes = out["cluster_id"].value_counts().rename_axis("cluster_id").reset_index(name="count")
cluster_summary = sizes.merge(summary, on="cluster_id", how="left")
cluster_summary.sort_values("count", ascending=False, inplace=True)
cluster_summary.to_csv("cluster_summary.csv", index=False)

print("\nCluster Summary (top 8):")
print(cluster_summary.head(8).to_string(index=False))

# Save unsupervised learning to pickle files
joblib.dump(scaler, "unsup_scaler.pkl")
joblib.dump(pca, "unsup_pca.pkl")
joblib.dump(kmeans, "unsup_kmeans.pkl")
print("\nSaved: unsup_scaler.pkl, unsup_pca.pkl, unsup_kmeans.pkl")


PCA components: 30
Explained variance (first 5 PCs cumulative): [0.12836405 0.17675469 0.21865164 0.25765873 0.28763853]
Components to reach ~99% variance: 30
k= 6 -> silhouette: 0.0725
k= 8 -> silhouette: 0.0678
k=10 -> silhouette: 0.0611
k=12 -> silhouette: 0.0627

Selected k=6 (silhouette=0.0725)


NameError: name 'agg_cols' is not defined

 PCA performing well is not surprising so many of the features we created were derivatives of stats that are already present. So it makes sense that PCA was able to compress these down into 30 features, there might only be 30 features that observe different qualities of an NBA game from the dataset.

 Cluster doees not seem to add much of anything. The silhouettes were incredibly small. We think this makes sense because the NBA is a multi-billion dollar industry and it makes sense that its converged similar team composition, style of play, and game outcomes.  Yes there is innovation and variance but teams are incentivized so heavily to win that we do not actually see these innovations completely.

In [ ]:
# Running the XGBoost with residual with the PCA treatment

import os
import json
import warnings
import numpy as np
import pandas as pd
import joblib
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

warnings.filterwarnings("ignore")

# Load data
print("\nLoading train and test data...")
train = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")

# Format Data
for df in (train, test):
    if 'GAME_DATE' in df.columns:
        df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'], errors='coerce')

## Columns
# Filter
metadata_cols = ["GAME_ID", "GAME_DATE", "TEAM_ID", "season"]

# Target
target_col = "actual_second_half_total"

# Residual
vegas_col = "h2_total"

# Features
feature_cols = [c for c in train.columns if c not in (metadata_cols + [target_col])]

# Split
X_train = train[feature_cols].copy()
X_test = test[feature_cols].copy()

y_train_total = train[target_col].values
y_test_total = test[target_col].values

# Residual targets
y_train = (train[target_col] - train[vegas_col]).values
y_test = (test[target_col]  - test[vegas_col]).values

# Impute with median this method is so much easier
medians = X_train.median(numeric_only=True)
X_train = X_train.fillna(medians)
X_test = X_test.fillna(medians)


# Load our scaler and pca
use_saved = False
scaler_path = "unsup_scaler.pkl"
pca_path = "unsup_pca.pkl"

if os.path.exists(scaler_path) and os.path.exists(pca_path):
    scaler = joblib.load(scaler_path)
    pca    = joblib.load(pca_path)
    # Verify feature count matches
    try:
        if hasattr(scaler, "n_features_in_") and scaler.n_features_in_ == X_train.shape[1]:
            use_saved = True
    except Exception:
        use_saved = False

if use_saved:
    print("\nUsing saved scaler & PCA (unsup_scaler.pkl / unsup_pca.pkl)")
else:
    print("Falling back to recreating scale and pca")

    scaler = StandardScaler(with_mean=True, with_std=True)
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Choose components: keep up to 99% variance, cap at 50 to be safe
    pca_full = PCA(svd_solver="auto", random_state=42)
    pca_full.fit(X_train_scaled)
    cumsum = np.cumsum(pca_full.explained_variance_ratio_)
    n_comp = int(np.searchsorted(cumsum, 0.99) + 1)
    n_comp = min(max(n_comp, 10), 50)  # between 10 and 50
    print(f"Selected PCA components: {n_comp} (≈99% variance)")

    pca = PCA(n_components=n_comp, svd_solver="auto", random_state=42)
    # Re-fit with chosen n_components
    X_train_scaled = scaler.fit_transform(X_train)
    pca.fit(X_train_scaled)

    # Save for reuse next time
    joblib.dump(scaler, scaler_path)
    joblib.dump(pca, pca_path)
    print("Saved new unsup_scaler.pkl and unsup_pca.pkl")

# Transform using the scaler/PCA in use (saved or fresh)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"\nPCA features shape -> train: {X_train_pca.shape}, test: {X_test_pca.shape}")

# Model with specs

model = xgb.XGBRegressor(
    n_estimators=1300,
    learning_rate=0.018,
    max_depth=6,
    min_child_weight=2,
    subsample=0.75,
    colsample_bytree=0.7,
    gamma=0.15,
    reg_alpha=0.6,
    reg_lambda=1.2,
    random_state=42,
    n_jobs=-1,
    early_stopping_rounds=75,
    eval_metric="rmse"
)

model.fit(
    X_train_pca, y_train,
    eval_set=[(X_train_pca, y_train), (X_test_pca, y_test)],
    verbose=50
)

# Eval
print("RESIDUAL PERFORMANCE (Model vs Vegas)")

y_pred_resid = model.predict(X_test_pca)

# Model Stats base
mae_resid = mean_absolute_error(y_test, y_pred_resid)
rmse_resid = np.sqrt(mean_squared_error(y_test, y_pred_resid))
r2_resid = r2_score(y_test, y_pred_resid)


# Reconstruct totals
y_pred_total = y_pred_resid + test[vegas_col].values
mae_total = mean_absolute_error(y_test_total, y_pred_total)
rmse_total = np.sqrt(mean_squared_error(y_test_total, y_pred_total))
r2_total = r2_score(y_test_total, y_pred_total)

# Vegas baseline on the same split
mae_line = mean_absolute_error(y_test_total, test[vegas_col].values)
rmse_line = np.sqrt(mean_squared_error(y_test_total, test[vegas_col].values))
r2_line = r2_score(y_test_total, test[vegas_col].values)

print("🔍 BENCHMARK vs VEGAS")
print(f"Vegas    -> MAE: {mae_line:.3f} | RMSE: {rmse_line:.3f} | R²: {r2_line:.3f}")
print(f"Residual -> MAE: {mae_total:.3f} | RMSE: {rmse_total:.3f} | R²: {r2_total:.3f}")
print(f"ΔMAE vs Vegas:  {mae_total - mae_line:+.3f}")
print(f"ΔRMSE vs Vegas: {rmse_total - rmse_line:+.3f}")

# Saves
test_out = test.copy()
test_out["predicted_residual_pca"] = y_pred_resid
test_out["predicted_total_pca"] = y_pred_total
test_out["edge_vs_vegas_pca"] = y_pred_resid
test_out["abs_error_pca"] = np.abs(y_test_total - y_pred_total)

test_out.to_csv("test_predictions_residual_pca.csv", index=False)
model.save_model("xgboost_residual_pca.json")

# Save another feature list
with open("pca_feature_cols.json", "w") as f:
    json.dump(feature_cols, f, indent=2)

print("Saved test_predictions_residual_pca.csv")
print("Saved xgboost_residual_pca.json")
print("Saved pca_feature_cols.json")



Loading train and test data...

Using saved scaler & PCA (unsup_scaler.pkl / unsup_pca.pkl)

PCA features shape -> train: (4988, 50), test: (1328, 50)
[0]	validation_0-rmse:12.93217	validation_1-rmse:13.18861
[50]	validation_0-rmse:12.24320	validation_1-rmse:13.21503
[75]	validation_0-rmse:11.91613	validation_1-rmse:13.23899
RESIDUAL PERFORMANCE (Model vs Vegas)
🔍 BENCHMARK vs VEGAS
Vegas    -> MAE: 10.299 | RMSE: 13.215 | R²: 0.121
Residual -> MAE: 10.287 | RMSE: 13.189 | R²: 0.125
ΔMAE vs Vegas:  -0.012  (negative is better)
ΔRMSE vs Vegas: -0.027 (negative is better)
Saved test_predictions_residual_pca.csv
Saved xgboost_residual_pca.json
Saved pca_feature_cols.json


In [9]:
"""
Find where h2_total ends and analyze joins for that specific range
"""

import pandas as pd
import numpy as np

print("="*60)
print("LOADING DATA")
print("="*60)

# Load datasets
games = pd.read_csv('nba_games_2021_to_2024.csv')
betting = pd.read_csv('betting_data.csv')

# Convert dates
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
betting['date'] = pd.to_datetime(betting['date'])

nba_min = games['GAME_DATE'].min()
nba_max = games['GAME_DATE'].max()

print(f"\nNBA Games: {games.shape}")
print(f"  Date range: {nba_min} to {nba_max}")

print(f"\nBetting Data: {betting.shape}")
print(f"  Date range: {betting['date'].min()} to {betting['date'].max()}")

# ============================================================================
# FIND WHERE H2_TOTAL DATA ENDS
# ============================================================================
print("\n" + "="*60)
print("ANALYZING H2_TOTAL AVAILABILITY")
print("="*60)

# Filter to betting data with h2_total only
betting_with_h2 = betting[betting['h2_total'].notna()].copy()

print(f"\nBetting rows with h2_total: {len(betting_with_h2):,} / {len(betting):,}")
print(f"  h2_total date range: {betting_with_h2['date'].min()} to {betting_with_h2['date'].max()}")

# Filter to NBA date range
betting_h2_in_nba_range = betting_with_h2[
    (betting_with_h2['date'] >= nba_min) &
    (betting_with_h2['date'] <= nba_max)
].copy()

print(f"\nBetting with h2_total in NBA date range: {len(betting_h2_in_nba_range):,}")
print(f"  Date range: {betting_h2_in_nba_range['date'].min()} to {betting_h2_in_nba_range['date'].max()}")

# Group by month to see when h2_total stops
betting_h2_in_nba_range['year_month'] = betting_h2_in_nba_range['date'].dt.to_period('M')
monthly_h2 = betting_h2_in_nba_range.groupby('year_month').size()

print(f"\nh2_total availability by month:")
print(monthly_h2)

# Find the last date with h2_total
h2_total_end_date = betting_h2_in_nba_range['date'].max()
h2_total_start_date = betting_h2_in_nba_range['date'].min()

print(f"\n🎯 H2_TOTAL DATE RANGE:")
print(f"  Start: {h2_total_start_date}")
print(f"  End: {h2_total_end_date}")
print(f"  Duration: {(h2_total_end_date - h2_total_start_date).days} days")

# ============================================================================
# FILTER NBA GAMES TO H2_TOTAL RANGE
# ============================================================================
print("\n" + "="*60)
print("FILTERING NBA GAMES TO H2_TOTAL RANGE")
print("="*60)

games_in_h2_range = games[
    (games['GAME_DATE'] >= h2_total_start_date) &
    (games['GAME_DATE'] <= h2_total_end_date)
].copy()

print(f"\nOriginal NBA games: {len(games):,}")
print(f"NBA games in h2_total range: {len(games_in_h2_range):,}")
print(f"Excluded (outside h2_total range): {len(games) - len(games_in_h2_range):,}")

# ============================================================================
# TEAM MAPPING
# ============================================================================
print("\n" + "="*60)
print("TEAM MAPPING")
print("="*60)

team_map = {
    'atl': 'ATL', 'bos': 'BOS', 'bkn': 'BKN', 'cha': 'CHA', 'chi': 'CHI',
    'cle': 'CLE', 'dal': 'DAL', 'den': 'DEN', 'det': 'DET', 'gs': 'GSW',
    'hou': 'HOU', 'ind': 'IND', 'lac': 'LAC', 'lal': 'LAL', 'mem': 'MEM',
    'mia': 'MIA', 'mil': 'MIL', 'min': 'MIN', 'no': 'NOP', 'nyk': 'NYK',
    'okc': 'OKC', 'orl': 'ORL', 'phi': 'PHI', 'phx': 'PHX', 'por': 'POR',
    'sac': 'SAC', 'sa': 'SAS', 'tor': 'TOR', 'utah': 'UTA', 'wsh': 'WAS',
    'nj': 'BKN', 'ny': 'NYK'
}

betting_h2_in_nba_range['away_team'] = betting_h2_in_nba_range['away'].map(team_map)
betting_h2_in_nba_range['home_team'] = betting_h2_in_nba_range['home'].map(team_map)

unmapped = betting_h2_in_nba_range[
    betting_h2_in_nba_range['away_team'].isna() | 
    betting_h2_in_nba_range['home_team'].isna()
]
print(f"Unmapped teams: {len(unmapped)}")

# ============================================================================
# CREATE MATCHUPS
# ============================================================================
print("\n" + "="*60)
print("CREATING MATCHUPS")
print("="*60)

# NBA matchups
games_in_h2_range['is_home'] = games_in_h2_range['MATCHUP'].str.contains('vs.', na=False)

home_games = games_in_h2_range[games_in_h2_range['is_home'] == True][
    ['GAME_ID', 'GAME_DATE', 'TEAM_ABBREVIATION']
].copy()
away_games = games_in_h2_range[games_in_h2_range['is_home'] == False][
    ['GAME_ID', 'GAME_DATE', 'TEAM_ABBREVIATION']
].copy()

home_games.columns = ['GAME_ID', 'GAME_DATE', 'home_team']
away_games.columns = ['GAME_ID', 'GAME_DATE', 'away_team']

nba_matchups = home_games.merge(away_games, on=['GAME_ID', 'GAME_DATE'])

print(f"NBA matchups: {len(nba_matchups):,}")

# Create match keys
nba_matchups['match_key'] = (
    nba_matchups['GAME_DATE'].dt.strftime('%Y-%m-%d') + '_' + 
    nba_matchups['away_team'] + '_' + 
    nba_matchups['home_team']
)

betting_h2_in_nba_range['match_key'] = (
    betting_h2_in_nba_range['date'].dt.strftime('%Y-%m-%d') + '_' + 
    betting_h2_in_nba_range['away_team'] + '_' + 
    betting_h2_in_nba_range['home_team']
)

# ============================================================================
# JOIN ANALYSIS
# ============================================================================
print("\n" + "="*60)
print("JOIN ANALYSIS")
print("="*60)

nba_keys = set(nba_matchups['match_key'])
betting_keys = set(betting_h2_in_nba_range['match_key'].dropna())

matched = nba_keys & betting_keys
nba_only = nba_keys - betting_keys
betting_only = betting_keys - nba_keys

print(f"\n📊 Match Results:")
print(f"  NBA games (in h2_total range): {len(nba_keys):,}")
print(f"  Betting lines with h2_total: {len(betting_keys):,}")
print(f"  ✅ Matched: {len(matched):,}")
print(f"  ⚠️  NBA games without h2_total: {len(nba_only):,}")
print(f"  ⚠️  Betting h2_total without NBA: {len(betting_only):,}")

match_rate = len(matched) / len(nba_keys) * 100 if len(nba_keys) > 0 else 0

print(f"\n📈 Match Rate: {match_rate:.1f}%")
print(f"   ({len(matched):,} / {len(nba_keys):,} NBA games have h2_total)")

# ============================================================================
# JOIN AND SAVE
# ============================================================================
print("\n" + "="*60)
print("SAVING RESULTS")
print("="*60)

# Perform join
nba_with_h2 = nba_matchups.merge(
    betting_h2_in_nba_range[['match_key', 'h2_total', 'h2_spread', 'spread', 'total']],
    on='match_key',
    how='left'
)

# Save main dataset
nba_with_h2.to_csv('nba_games_with_h2_total.csv', index=False)
print(f"✅ Saved nba_games_with_h2_total.csv")
print(f"   Total games: {len(nba_with_h2):,}")
print(f"   With h2_total: {nba_with_h2['h2_total'].notna().sum():,}")
print(f"   Without h2_total: {nba_with_h2['h2_total'].isna().sum():,}")

# Save failed NBA joins (NBA games that should have h2_total but don't)
if len(nba_only) > 0:
    failed_nba = nba_matchups[nba_matchups['match_key'].isin(nba_only)]
    failed_nba.to_csv('nba_games_missing_h2_total.csv', index=False)
    print(f"\n❌ Saved nba_games_missing_h2_total.csv")
    print(f"   NBA games without h2_total: {len(failed_nba):,}")
    
    # Show sample
    print(f"\n   Sample games missing h2_total:")
    print(failed_nba[['GAME_DATE', 'away_team', 'home_team', 'GAME_ID']].head(10))
    
    # Group by date to see patterns
    failed_by_date = failed_nba.groupby(failed_nba['GAME_DATE'].dt.date).size()
    print(f"\n   Missing h2_total by date (first 20 days):")
    print(failed_by_date.head(20))

# ============================================================================
# SUMMARY
# ============================================================================
print("\n" + "="*60)
print("SUMMARY")
print("="*60)

print(f"\n🎯 H2_TOTAL Coverage:")
print(f"  Date range analyzed: {h2_total_start_date} to {h2_total_end_date}")
print(f"  NBA games in range: {len(nba_matchups):,}")
print(f"  Games with h2_total: {len(matched):,} ({match_rate:.1f}%)")
print(f"  Games missing h2_total: {len(nba_only):,} ({len(nba_only)/len(nba_matchups)*100:.1f}%)")

print(f"\n📁 Files Created:")
print(f"  - nba_games_with_h2_total.csv (use this for modeling)")
print(f"  - nba_games_missing_h2_total.csv (failed joins to investigate)")

print(f"\n💡 Next Step:")
print(f"   Review nba_games_missing_h2_total.csv to understand why {len(nba_only)} games")
print(f"   within the h2_total date range don't have betting lines")

LOADING DATA

NBA Games: (15615, 28)
  Date range: 2020-12-11 00:00:00 to 2024-06-17 00:00:00

Betting Data: (23118, 27)
  Date range: 2007-10-30 00:00:00 to 2025-06-22 00:00:00

ANALYZING H2_TOTAL AVAILABILITY

Betting rows with h2_total: 19,817 / 23,118
  h2_total date range: 2007-10-30 00:00:00 to 2023-01-16 00:00:00

Betting with h2_total in NBA date range: 3,158
  Date range: 2020-12-22 00:00:00 to 2023-01-16 00:00:00

h2_total availability by month:
year_month
2020-12     67
2021-01    222
2021-02    212
2021-03    204
2021-04    240
2021-05    173
2021-06     45
2021-07      8
2021-10     93
2021-11    225
2021-12    209
2022-01    231
2022-02    163
2022-03    229
2022-04    129
2022-05     38
2022-06      6
2022-10    102
2022-11    222
2022-12    222
2023-01    118
Freq: M, dtype: int64

🎯 H2_TOTAL DATE RANGE:
  Start: 2020-12-22 00:00:00
  End: 2023-01-16 00:00:00
  Duration: 755 days

FILTERING NBA GAMES TO H2_TOTAL RANGE

Original NBA games: 15,615
NBA games in h2_total ra

In [ ]:
# Predict Holdouts
import pandas as pd
import numpy as np
import xgboost as xgb

# Loading Model
model = xgb.XGBRegressor()
model.load_model("xgboost_residual_model.json")

# Load Data
holdout_data = pd.read_csv("holdout_data.csv")
print(f"Holdout: {len(holdout_data)} rows")

# Features
metadata_cols = ["GAME_ID", "GAME_DATE", "TEAM_ID", "season"]
target_col = "actual_second_half_total"
vegas_col = "h2_total"

feature_cols = [c for c in holdout_data.columns if c not in metadata_cols + [target_col]]
X_holdout = holdout_data[feature_cols].copy()

# Impute missing values
for col in X_holdout.columns:
    if X_holdout[col].isnull().any():
        X_holdout[col].fillna(X_holdout[col].median(), inplace=True)

# Predict
predicted_residuals = model.predict(X_holdout)

# Write predictions to DF
holdout_data["predicted_residual"] = predicted_residuals
holdout_data["predicted_total"] = predicted_residuals + holdout_data[vegas_col]
holdout_data["bet_edge"] = predicted_residuals

# Save complete dataset
holdout_data.to_csv("holdout_predictions.csv", index=False)
print(f"Saved holdout_predictions.csv")

## Clean up for a streamlined df.

# Select key columns
clean_predictions = holdout_data[[
    'GAME_ID',
    'GAME_DATE',
    'TEAM_ID',
    'h2_total',
    'predicted_total',
    'actual_second_half_total',
    'bet_edge'
]].copy()

# Format dates
clean_predictions['GAME_DATE'] = pd.to_datetime(clean_predictions['GAME_DATE']).dt.strftime('%Y-%m-%d')

# Rename columns
clean_predictions.columns = [
    'game_id',
    'date',
    'team_id',
    'h2_total',
    'predicted_total',
    'actual_total',
    'bet_edge'
]

# Sort by date
clean_predictions = clean_predictions.sort_values('date')

# Save clean (two rows per game - home and away teams)
clean_predictions.to_csv('holdout_predictions_clean.csv', index=False)
print(f"Saved holdout_predictions_clean.csv")

# Save game-level (one row per game)
game_level = clean_predictions.drop_duplicates('game_id').copy()
game_level.to_csv('holdout_predictions_games.csv', index=False)
print(f"Saved holdout_predictions_games.csv")

Loading model and holdout data...
Holdout: 1328 rows

Predicting...
✅ Saved holdout_predictions.csv ((1328, 1341))

Cleaning predictions...
✅ Saved holdout_predictions_clean.csv ((1328, 7))
✅ Saved holdout_predictions_games.csv ((664, 7))

🎉 Done!

📊 Summary:
   Total predictions: 1328
   Unique games: 664
   Games with h2_total: 664
   Games with actuals: 664


In [62]:
# Finding betting thresholds
import pandas as pd
import numpy as np

# Use games data
holdout_data = pd.read_csv("holdout_predictions_games.csv")

# Filter to games with both actuals and betting lines
bettable = holdout_data[
    (holdout_data["h2_total"].notna()) &
    (holdout_data["actual_total"].notna())
].copy()

# Comparison cols
bet_edge = bettable["bet_edge"].values
actual_total = bettable["actual_total"].values  # Changed
vegas_line = bettable["h2_total"].values

# Results dict
results = []

# Testing a thresholds in a range .5 all the way to 7.25 in .25 increments
thresholds = np.arange(0.5, 7.25, 0.25)

for threshold in thresholds:
    # Filtering bets
    bet_over = bet_edge > threshold
    bet_under = bet_edge < -threshold
    
    # Count bets
    over_count = bet_over.sum()
    under_count = bet_under.sum()
    total_bets = over_count + under_count
    
    if total_bets == 0:
        continue
    
    # Calculate wins
    over_wins = ((actual_total > vegas_line) & bet_over).sum()
    under_wins = ((actual_total < vegas_line) & bet_under).sum()
    total_wins = over_wins + under_wins
    
    # Win rates
    over_win_rate = over_wins / over_count * 100 if over_count > 0 else 0
    under_win_rate = under_wins / under_count * 100 if under_count > 0 else 0
    total_win_rate = total_wins / total_bets * 100
    
    # Profit with 110 odds
    over_profit = (over_wins * 100) - ((over_count - over_wins) * 110)
    under_profit = (under_wins * 100) - ((under_count - under_wins) * 110)
    total_profit = over_profit + under_profit
    
    # ROI
    total_risked = total_bets * 110
    roi = (total_profit / total_risked * 100) if total_risked > 0 else 0
    
    results.append({
        'threshold': threshold,
        'total_bets': total_bets,
        'over_bets': over_count,
        'under_bets': under_count,
        'total_wins': total_wins,
        'over_wins': over_wins,
        'under_wins': under_wins,
        'win_rate': total_win_rate,
        'over_win_rate': over_win_rate,
        'under_win_rate': under_win_rate,
        'profit': total_profit,
        'roi': roi
    })

# CAst to DF
results_df = pd.DataFrame(results)

# prints
top_roi = results_df.nlargest(10, 'roi')
print(top_roi[['threshold', 'total_bets', 'win_rate', 'profit', 'roi']].to_string(index=False))

# Write results
results_df.to_csv('threshold_optimization_results.csv', index=False)
print(f"Saved threshold_optimization_results.csv")

 threshold  total_bets  win_rate  profit        roi
      1.25          81 51.851852     -90  -1.010101
      1.50          46 50.000000    -230  -4.545455
      0.50         335 45.074627   -5140 -13.948440
      2.00          18 44.444444    -300 -15.151515
      1.00         149 44.295302   -2530 -15.436242
      0.75         234 44.017094   -4110 -15.967366
      1.75          30 43.333333    -570 -17.272727
      2.50           8 37.500000    -250 -28.409091
      2.75           8 37.500000    -250 -28.409091
      2.25          11 36.363636    -370 -30.578512
Saved threshold_optimization_results.csv
